# **Competencia 2 - CC6205 Natural Language Processing 📚**

Integrantes: Sebastián Valdivia - Patricio Ortiz - Marcelo Fuentealba

Fecha límite de entrega 📆: 29 de Junio.

Link competencia: https://codalab.lisn.upsaclay.fr/competitions/5098?secret_key=09955d45-6210-4a35-a171-8050aa050855#results

### **Objetivo**

El objetivo de esta competencia es resolver una de las tareas más importantes en el área del procesamiento de lenguage natural, relacionada con la extracción de información: [Named Entity Recognition (NER)](http://www.cs.columbia.edu/~mcollins/cs4705-spring2019/slides/tagging.pdf). 

En particular, y al igual que en la competencia anterior, deberán crear distintos modelos que apunten a resolver la tarea de NER en Español. Para esto, les entregaremos un dataset real perteneciente a la lista de espera NO GES en Chile. Es importante destacar que existe una falta de trabajos realizados en el área de NER en Español y aún más en el contexto clínico, por ende puede ser considerado como una tarea bien desafiante y quizás les interesa trabajar en el área más adelante en sus carreras.

En este notebook les entregaremos un baseline como referencia de los resultados que esperamos puedan obtener. Recuerden que el no superar a los baselines en alguna de las tres métricas conlleva un descuento de 0.5 puntos hasta 1.5 puntos.

Como hemos estado viendo redes neuronales tanto en catedras, tareas y auxiliares (o próximamente lo harán), esperamos que (por lo menos) utilicen Redes Neuronales Recurrentes (RNN) para resolverla. 

Nuevamente, hay total libertad para utilizar el software y los modelos que deseen, siempre y cuando estos no traigan los modelos ya implementados. (De todas maneras como es un corpus nuevo, es difícil que haya algún modelo ya implementado con estas entidades)

### **Explicación de la competencia**

La tarea **NER** que van a resolver en esta competencia es comúnmente abordada como un problema de Sequence Labeling.

**¿Qué es Sequence Labeling?** 

En breves palabras, dada una secuencia de tokens (oración) sequence labeling tiene por objetivo asignar una etiqueta a cada token de dicha secuencia. En pocas palabras, dada una lista de tokens esperamos encontrar la mejor secuencia de etiquetas asociadas a esa lista. Ahora veamos de qué se trata este problema.

**Named Entity Recognition (NER)**

NER es un ejemplo de un problema de Sequence Labeling. Pero antes de definir formalmente esta tarea, es necesario definir algunos conceptos claves para poder entenderla de la mejor manera:

- *Token*: Un token es una secuencia de caracteres, puede ser una palabra, un número o un símbolo.

- *Entidad*: No es más que un trozo de texto (uno o más tokens) asociado a una categoría predefinida. Originalmente se solían utilizar categorías como nombres de personas, organizaciones, ubicaciones, pero actualmente se ha extendido a diferentes dominios.

- *Límites de una entidad*: Son los índices de los tokens de inicio y fín dentro de una entidad.

- *Tipo de entidad*: Es la categoría predefinida asociada a la entidad.

Dicho esto, definimos formalmente una entidad como una tupla: $(s, e, t)$, donde $s, t$ son los límites de la entidad (índices de los tokens de inicio y fin, respectivamente) y t corresponde al tipo de entidad o categoría. Ya veremos más ejemplos luego de describir el Dataset.

**Corpus de la Lista de espera**

Trabajaran con un conjunto de datos reales correspondiente a interconsultas de la lista de espera NO GES en Chile. Si quieren saber más sobre cómo fueron generados los datos pueden revisar el paper publicado hace unos meses atrás en el workshop de EMNLP, una de las conferencias más importantes de NLP: [https://www.aclweb.org/anthology/2020.clinicalnlp-1.32/](https://www.aclweb.org/anthology/2020.clinicalnlp-1.32/).

Este corpus Chileno está constituido originalmente por 7 tipos de entidades pero por simplicidad en esta competencia trabajarán con las siguientes:

- **Disease**
- **Body_Part**
- **Medication** 
- **Procedures** 
- **Family_Member**

Si quieren obtener más información sobre estas entidades pueden consultar la [guía de anotación](https://plncmm.github.io/annodoc/). Además, mencionar que este corpus está restringido bajo una licencia que permite solamente su uso académico, así que no puede ser compartido más allá de este curso o sin permisos por parte de los autores en caso que quieran utilizarlo fuera. Si este último es el caso entonces pueden escribir directamente al correo: pln@cmm.uchile.cl. Al aceptar los términos y condiciones de la competencia están de acuerdo con los puntos descritos anteriormente.


**Formato ConLL**

Los archivos que serán entregados a ustedes vienen en un formato estándar utilizado en NER, llamado ConLL. No es más que un archivo de texto, que cumple las siguientes propiedades.

- Un salto de linea corresponde a la separación entre oraciones. Esto es importante ya que al entrenar una red neuronal ustedes pasaran una lista de oraciones como input, más conocidos como batches.

- La primera columna del archivo contiene todos los tokens de la partición.

- La segunda columna del archivo contiene el tipo de entidad asociado al token de la primera columna.

- Los tipos de entidades siguen un formato clásico en NER denominado *IOB2*. Si un tipo de entidad comienza con el prefijo "B-" (Beginning) significa que es el token de inicio de una entidad, si comienza con "I-" (Inside) es un token distinto al de inicio y si un token está asociado a la categoría O (Outside) significa que no pertenece a ninguna entidad.

Aquí va un ejemplo:

```
PACIENTE O
PRESENTA O
FRACTURA B-Disease
CORONARIA I-Disease
COMPLICADA I-Disease
EN O
PIE B-Body_Part
IZQUIERDO I-Body_Part
. O
SE O
REALIZA O
INSTRUMENTACION B-Procedure
INTRACONDUCTO I-Procedure
. O
```

Según nuestra definición tenemos las siguientes tres entidades (enumerando desde 0): 

- $(2, 4, Disease)$
- $(6, 7, Body Part)$
- $(11, 12, Procedure)$

Repasen un par de veces todos estos conceptos antes de pasar a la siguiente sección del notebook.
Es importante entender bien este formato ya que al medir el rendimiento de sus modelos, consideraremos una **métrica estricta**. Esta métrica se llama así ya que considera correcta una predicción de su modelo, sólo si al compararlo con las entidades reales **coinciden tanto los límites de la entidad como el tipo.** 

Para ejemplificar, tomando el caso anterior, si el modelo es capaz de encontrar la siguiente entidad: $(2, 3, Disease)$, entonces se considera incorrecto ya que pudo predecir dos de los tres tokens de dicha enfermedad. Es decir, buscamos una métrica que sea alta a nivel de entidad y no a nivel de token.

Antes de pasar a explicar las reglas, se recomienda visitar los siguientes links para entender bien el baseline de la competencia:

-  [Tagging, and Hidden Markov Models ](http://www.cs.columbia.edu/~mcollins/cs4705-spring2019/slides/tagging.pdf) (slides by Michael Collins), [notes](http://www.cs.columbia.edu/~mcollins/hmms-spring2013.pdf), [video 1](https://youtu.be/-ngfOZz8yK0), [video 2](https://youtu.be/PLoLKQwkONw), [video 3](https://youtu.be/aaa5Qoi8Vco), [video 4](https://youtu.be/4pKWIDkF_6Y)       
-  [Recurrent Neural Networks](slides/NLP-RNN.pdf) | [video 1](https://youtu.be/BmhjUkzz3nk), [video 2](https://youtu.be/z43YFR1iIvk), [video 3](https://youtu.be/7L5JxQdwNJk)


Recuerden que todo el material se encuentra disponible en el [github del curso](https://github.com/dccuchile/CC6205).

### **Reglas de la competencia**

- Para que su competencia sea evaluada, deben participar en la competencia y enviar este notebook con su informe.
- Para participar, deben registrarse en la competencia en Codalab en grupos de máximo 3 alumnos. Cada grupo debe tener un nombre de equipo. (¡Y deben reportarlo en su informe, por favor!)
- Las métricas usadas serán métricas estrictas (ya explicado anteriormente) utilizando métricas clásicas como lo son precisión, recall y micro f1-score.
- En esta tarea se recomienda usar GPU. Pueden ejecutar su tarea en colab (lo cual trae todo instalado) o pueden intentar ejecutándolo en su computador. En este caso, deberá ser compatible con cuda y deberán instalar todo por su cuenta.
- En total pueden hacer un **máximo de 5 envíos**.
- Por favor, todas sus dudas haganlas por el canal de Discord. Los emails que lleguen al equipo docente serán remitidos a ese medio. Recuerden el ánimo colaborativo del curso.
- Estar top 5 en alguna de las tres métricas equivale a una bonificación en su nota final.

Éxito!


### **Baseline**

En este punto esperamos que tengan conocimiento sobre redes neuronales y en particular redes neuronales recurrentes (RNN), si no siempre pueden escribirnos por el canal de Discord para aclarar dudas. La RNN del baseline adjunto a este notebook está programado en la librería [`pytorch`](https://pytorch.org/) pero ustedes pueden utilizar keras, tensorflow si así lo desean. El código contiene lo siguiente:

- La carga de los datasets, creación de batches de texto y padding (esto es importante ya que si utilizan redes neuronales tienen que tener el mismo largo los inputs). 

- La implementación básica de una red `LSTM` simple de solo un nivel y sin bi-direccionalidad. 

- La construcción del formato del output requerido para que lo puedan probar en la tarea en codalab.

Se espera que como mínimo ustedes puedan experimentar con el baseline utilizando (pero no limitándose) estas sugerencias:

*   Probar la técnica de early stopping.
*   Variar la cantidad de parámetros de la capa de embeddings.
*   Variar la cantidad de capas RNN.
*   Variar la cantidad de parámetros de las capas de RNN.
*   Inicializar la capa de embeddings con modelos pre-entrenados. (word2vec, glove, conceptnet, etc...). [Embeddings en español aquí](https://github.com/dccuchile/spanish-word-embeddings). También aquí pueden encontrar unos embeddings clínicos en Español: [https://zenodo.org/record/3924799](https://zenodo.org/record/3924799)
*   Variar la cantidad de épocas de entrenamiento.
*   Variar el optimizador, learning rate, batch size, usar CRF loss, etc.
*   Probar una capa de CRF para garantizar el     formato IOB2.
*   Probar bi-direccionalidad.
*   Incluir dropout.
*   Probar modelos de tipo GRU.
*   Probar usando capas de atención.
*   Probar Embedding Contextuales (les puede ser de utilidad [flair](https://github.com/flairNLP/flair))
*   Probar modelos de transformers en español usando [Huggingface](https://github.com/huggingface/transformers) o el framework Flair.

### **Reporte**

Este debe cumplir la siguiente estructura:

1.	**Introducción**: Presentar brevemente el contexto, problema a resolver, incluyendo la formalización de la task (cómo son los inputs y outputs del problema) y los desafíos que ven al analizar el corpus entregado. (**0.5 puntos**)

2.	**Modelos**: Describir brevemente los modelos, métodos e hiperparámetros utilizados. (**1.0 puntos**)

4.	**Métricas de evaluación**: Describir las métricas utilizadas en la evaluación indicando qué miden y cuál es su interpretación en este problema en particular. (**0.5 puntos**)

5.  **Diseño experimental**: Esta es una de las secciones más importantes del reporte. Deben describir minuciosamente los experimentos que realizarán en la siguiente sección. Describir las variables de control que manejarán, algunos ejemplos pueden ser: Los hiperparámetros de los modelos, tipo de embeddings utilizados, tipos de arquitecturas. Ser claros con el conjunto de hiperparámetros que probarán, la decisión en las funciones de optimización, función de pérdida,  regulación, etc. Básicamente explicar qué es lo que veremos en la siguiente sección.
(**1 punto**)

6.	**Experimentos**: Reportar todos sus experimentos y código en esta sección. Comparar los resultados obtenidos utilizando diferentes modelos. ¡Es vital haber realizado varios experimentos para sacar una buena nota! (**2.0 puntos**)

7.	**Conclusiones**: Discutir resultados, proponer trabajo futuro. (**1 punto**)

# **Entregable.**

## **Introducción**


Esta competencia se da en el contexto de las listas de espera NO GES en Chile, en particular, se debe resolver la tarea de Named Entity Recognition (NER) para un dataset que contiene el texto anotado en fichas clínicas de interconsultas de pacientes. Las categorías de entidad que se disponen para etiquetar corresponden a:
* Disease: Enfermedades mapeables a un código CIE-10.
* Body_Part: Órgano o parte anatómica de una persona.
* Medication: Todas las menciones de medicamentos o drogas empleadas.
* Procedures: Procedimientos diagnósticos, terapéuticos y de laboratorio.
* Family_Member: Etiqueta para miembros de la familia.

Los inputs están en formato ConLL, los cuales deben ser procesados y adaptados para hacer una adecuada lectura de ellos. Visto de manera simple, un input corresponde a una oración que contiene entidades.
El output del problema consiste en un vector que contiene la etiqueta de las entidades correspondientes al input entregado.

Los desafíos que se enfrentan en la task se centran principalmente montar la infraestructura necesaria para que los modelos sean capaces de mantener "memoria" y de esta manera lograr hacer un buen NER.


## **Modelos**


Para abordar este trabajo se utilizan 3 modelos de redes neuronales y finalmente a cada uno se le aplica bidireccionalidad.

En primer lugar, se trabaja con LSTM que corresponden a un tipo específico de redes neuronales recurrentes. Se caracterizan por tener la capacidad de "recordar" estados previos y utilizar dicha información para definir el próximo estado. Para el caso específico de NER son útiles porque pueden aprender dependencias y no solo de corto plazo, de modo que se evita el supuesto de independencia que se hace, por ejemplo, en los modelos HMM.
MÉTODOS, HIPERPARÁMETROS
Para este caso, los hiperparámetros corresponden a la dimensión de los embeddings, dimensión de las capas LSTM, el número de capas, dropout, número de épocas.


En segundo lugar, se trabaja con redes Elman, caracterizadas por generar y detectar patrones variantes, y también es capaz de retener información de estados previos.
MÉTODOS, HIPERPARÁMETROS
En este caso, los hiperparámetros son la dimensión de los embeddings, dimensión de las capas LSTM, el número de capas, dropout.

En tercer lugar, se utilizan redes tipo GRU que corresponde a una variante y simplificación de la LSTM. GRU también puede resolver el problema de dependencia de largo plazo.
MÉTODOS, HIPERPARÁMETROS
En este caso, los hiperparámetros son la dimensión de los embeddings, dimensión de las capas LSTM, el número de capas, dropout.

Por último, se toma cada uno de los modelos y se implementa bidireccionalidad, con el fin de incorporar la capacidad de acceder a contextos futuros y no solo pasados, aspecto que es candidato a mejorar el rendimiento de los modelos en la task. Para este caso, los hiperparámetros se mantienen según sea el modelo y se específica adicionalmente que existe bidireccionalidad.

## **Métricas de evaluación**



- **Métrica estricta:** Cuando se habla de métrica estricta se considera que un documento está bien clasificado si cada una de las etiquetas predichas por el modelo son correctas, en caso contrario (al menos una etiqueta errónea), se considera mala clasificación.

- **Precision:** Indica qué proporción de los documentos clasificados como positivos, por el modelo, son realmente son positivos. Responde a la pregunta ¿qué porcentaje de los que hemos dicho que son de clase positiva, en realidad lo son? (TP/(TP+FP))

 

- **Recall:** Indica qué proporción de los documentos positivos fueron clasificados como positivos por el modelo. Responde a la pregunta ¿qué porcentaje de la clase positiva hemos sido capaces de identificar correctamente? (TP/(TP+FN))


- **Micro F1 score:**
PENDIENTE Recuerde hacer la distinción entre lo que sería una métrica de micro f1-score vs macro f1-score.

## **Diseño experimental**

###Early Stopping
Para la primera sección de experimentos se implementa Early Stopping, ya que con esta técnica es posible experimentar sobre la mayor cantidad de épocas sobre los modelos sin tener que preocuparnos por el overfitting.
Sin un mecanismo como este, es posible que la red termine sobreajustándose a los datos de entrenamiento si la sesión de entrenamiento no se detiene en el instante correcto, de modo que la red adquiere ruido y no se hace generalizable.

###Redes Elman y GRU
Para la siguiente experimentación se barajan alternativas a las redes recurentes LSTM que venía como baseline del proyecto, por lo que se procede a experimentar con modelos GRU y Elmann unidireccionales, ya que estas ya se encuentran implementadas en Pytorch, además de utilizar la función earlyStop_model para evitar el overfitting.

###Redes Bidireccionales
En este caso se tomarán las redes previamente trabajadas y se implementará la bidireccionalidad, aspecto interesante, ya que se trata de la combinación de dos redes: una que va de izquierda a derecha y otra que va de derecha a izquierda. Con esto es posible capturar el contexto de las oraciones considerando lo previo y lo posterior. Para NER, sabemos que el contexto cubre tags pasadas y futuras en una secuencia, por ello el hecho tener en cuenta tanto la información pasada como la futura podría agregar valor y mejorar el rendimiento de los modelos.

## **Experimentos**


El código que les entregaremos servirá de baseline para luego implementar mejores modelos. 
En general, el código asociado a la carga de los datos, las funciones de entrenamiento, de evaluación y la predicción de los datos de la competencia no deberían cambiar. 
Solo deben preocuparse de cambiar la arquitectura del modelo, sus hiperparámetros y reportar, lo cual lo pueden hacer en las subsecciones *modelos*.

Los modelos con los que se experimentará serán:

1. Modelos Elman y GRU: Se implementa versión unidireccional, con Early Stopping. Los hiperparametros usados son: Embedding de dimensión 300. Capas de LSTM de tamaño 256. Numero de capas = 3. Dropout de 0.3

2. Modelos LSTM, Elman y GRU: Se implementa versión bidireccional, con Early Stopping. Los hiperparametros usados son: Embedding de dimensión 300. Capas de LSTM de tamaño 256. Numero de capas = 3. Dropout de 0.3

3. Arquitecturas de LSTM bidireccional: Posterior a los ejercicios previos, se probaran distintas arquitecturas de LSTM's variando los hiperparametros.

Para todos los modelos se utiliza función de pérdida CrossEntropy y optimizador Adam.



###  **Carga de datos y Preprocesamiento**

Para cargar los datos y preprocesarlos usaremos la librería [`torchtext`](https://github.com/pytorch/text). Tener cuidado ya que hace algunos meses esta librería tuvo cambios radicales, quedando las funcionalidades pasadas en un nuevo paquete llamado legacy. Esto ya que si quieren usar más funciones de la librería entonces vean los cambios en la documentación.

En particular usaremos su módulo `data`, el cual según su documentación original provee: 

    - Ability to describe declaratively how to load a custom NLP dataset that's in a "normal" format
    - Ability to define a preprocessing pipeline
    - Batching, padding, and numericalizing (including building a vocabulary object)
    - Wrapper for dataset splits (train, validation, test)


El proceso será el siguiente: 

1. Descargar los datos desde github y examinarlos.
2. Definir los campos (`fields`) que cargaremos desde los archivos.
3. Cargar los datasets.
4. Crear el vocabulario.

In [ ]:
# Instalamos torchtext que nos facilitará la vida en el pre-procesamiento del formato ConLL.
!pip install -U torchtext==0.10.0

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import torch
from torchtext import data, datasets, legacy


# Garantizar reproducibilidad de los experimentos
SEED = 1234
torch.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

#### **Obtener datos**

Descargamos los datos de entrenamiento, validación y prueba en nuestro directorio de trabajo

In [ ]:
#%%capture

!wget https://github.com/dccuchile/CC6205/releases/download/v1.0/train.txt -nc # Dataset de Entrenamiento
!wget https://github.com/dccuchile/CC6205/releases/download/v1.0/dev.txt -nc    # Dataset de Validación (Para probar y ajustar el modelo)
!wget https://github.com/dccuchile/CC6205/releases/download/v1.0/test.txt -nc  # Dataset de la Competencia. Estos datos solo contienen los tokens. ¡¡SON LOS QUE DEBEN SER PREDICHOS!!

File ‘train.txt’ already there; not retrieving.

File ‘dev.txt’ already there; not retrieving.

File ‘test.txt’ already there; not retrieving.



####  **Fields**

Un `field`:

* Define un tipo de datos junto con instrucciones para convertir el texto a Tensor.
* Contiene un objeto `Vocab` que contiene el vocabulario (palabras posibles que puede tomar ese campo).
* Contiene otros parámetros relacionados con la forma en que se debe numericalizar un tipo de datos, como un método de tokenización y el tipo de Tensor que se debe producir.


Analizemos el siguiente cuadro el cual contiene un ejemplo cualquiera de entrenamiento:


```
El O
paciente O
padece O
de O
cancer B-Disease
de I-Disease
colon I-Disease
. O
```

Cada linea contiene un token y el tipo de entidad asociado en el formato IOB2 ya explicado. Para que `torchtext` pueda cargar estos datos, debemos definir como va a leer y separar los componentes de cada una de las lineas.
Para esto, definiremos un field para cada uno de esos componentes: Las palabras (`TEXT`) y las etiquetas o categorías (`NER_TAGS`).


In [ ]:
# Primer Field: TEXT. Representan los tokens de la secuencia
TEXT = legacy.data.Field(lower=False) 

# Segundo Field: NER_TAGS. Representan los Tags asociados a cada palabra.
NER_TAGS = legacy.data.Field(unk_token=None)
fields = (("text", TEXT), ("nertags", NER_TAGS))

In [ ]:
fields

(('text', <torchtext.legacy.data.field.Field at 0x7fcf91cf9450>),
 ('nertags', <torchtext.legacy.data.field.Field at 0x7fcfef8bebd0>))

####  **SequenceTaggingDataset**

`SequenceTaggingDataset` es una clase de torchtext diseñada para contener datasets de sequence labeling. Los ejemplos que se guarden en una instancia de estos serán arreglos de palabras asociados con sus respectivos tags.

Por ejemplo, para Part-of-speech tagging:

[I, love, PyTorch, .] estará asociado con [PRON, VERB, PROPN, PUNCT]


La idea es que usando los fields que definimos antes, le indiquemos a la clase cómo cargar los datasets de prueba, validación y test.

In [ ]:
train_data, valid_data, test_data = legacy.datasets.SequenceTaggingDataset.splits(
    path="./",
    train="train.txt",
    validation="dev.txt",
    test="test.txt",
    fields=fields,
    encoding="utf-8",
    separator=" "
)

In [ ]:
print(f"Numero de ejemplos de entrenamiento: {len(train_data)}")
print(f"Número de ejemplos de validación: {len(valid_data)}")
print(f"Número de ejemplos de test (competencia): {len(test_data)}")

Numero de ejemplos de entrenamiento: 8025
Número de ejemplos de validación: 891
Número de ejemplos de test (competencia): 992


Visualizemos un ejemplo

In [ ]:
import random
random_item_idx = random.randint(0, len(train_data))
random_example = train_data.examples[random_item_idx]
list(zip(random_example.text, random_example.nertags))

[('-', 'O'),
 ('DESDENTADO', 'B-Disease'),
 ('PARCIAL', 'I-Disease'),
 ('/', 'O'),
 ('-', 'O'),
 ('Fundamento', 'O'),
 ('Clínico', 'O'),
 ('APS', 'O'),
 (':', 'O'),
 ('Paciente', 'O'),
 ('desdentado', 'B-Disease'),
 ('parcial', 'I-Disease'),
 ('superior', 'I-Disease'),
 ('e', 'I-Disease'),
 ('inferior', 'I-Disease'),
 (',', 'O'),
 ('se', 'O'),
 ('solicita', 'O'),
 ('protesis', 'B-Procedure'),
 ('removible', 'I-Procedure'),
 ('superior', 'I-Procedure'),
 ('e', 'I-Procedure'),
 ('inferior', 'I-Procedure'),
 ('.', 'O')]

#### **Construir los vocabularios para el texto y las etiquetas**

Los vocabularios son los objetos que contienen todos los tokens (de entrenamiento) posibles para ambos fields. El siguiente paso consiste en construirlos. Para esto, hacemos uso del método `Field.build_vocab` sobre cada uno de nuestros `fields`. 

In [ ]:
TEXT.build_vocab(train_data)
NER_TAGS.build_vocab(train_data)

In [ ]:
print(f"Tokens únicos en TEXT: {len(TEXT.vocab)}")
print(f"Tokens únicos en NER_TAGS: {len(NER_TAGS.vocab)}")

Tokens únicos en TEXT: 17591
Tokens únicos en NER_TAGS: 12


In [ ]:
#Veamos las posibles etiquetas que hemos cargado:
NER_TAGS.vocab.itos

['<pad>',
 'O',
 'I-Disease',
 'B-Disease',
 'I-Body_Part',
 'B-Body_Part',
 'B-Procedure',
 'I-Procedure',
 'B-Medication',
 'B-Family_Member',
 'I-Medication',
 'I-Family_Member']

Observen que ademas de los tags NER, tenemos \<pad\>, el cual es generado por el dataloader para cumplir con el padding de cada oración.

Veamos ahora los tokens mas frecuentes y especiales:

In [ ]:
# Tokens mas frecuentes (Será necesario usar stopwords, eliminar símbolos o nos entregan información (?) )
TEXT.vocab.freqs.most_common(10)

[('.', 7396),
 (',', 6821),
 ('-', 4985),
 ('de', 3811),
 ('DE', 3645),
 ('/', 2317),
 (':', 2209),
 ('con', 1484),
 ('y', 1439),
 ('APS', 1429)]

In [ ]:
# Seteamos algunas variables que nos serán de utilidad mas adelante...
UNK_IDX = TEXT.vocab.stoi[TEXT.unk_token]
PAD_IDX = TEXT.vocab.stoi[TEXT.pad_token]

PAD_TAG_IDX = NER_TAGS.vocab.stoi[NER_TAGS.pad_token]
O_TAG_IDX = NER_TAGS.vocab.stoi['O']

#### **Frecuencia de los Tags**

Visualizemos rápidamente las cantidades y frecuencias de cada tag:

In [ ]:
def tag_percentage(tag_counts):
    
    total_count = sum([count for tag, count in tag_counts])
    tag_counts_percentages = [(tag, count, count/total_count) for tag, count in tag_counts]
  
    return tag_counts_percentages

print("Tag Ocurrencia Porcentaje\n")

for tag, count, percent in tag_percentage(NER_TAGS.vocab.freqs.most_common()):
    print(f"{tag}\t{count}\t{percent*100:4.1f}%")

Tag Ocurrencia Porcentaje

O	101671	68.1%
I-Disease	21629	14.5%
B-Disease	8831	 5.9%
I-Body_Part	6489	 4.3%
B-Body_Part	3755	 2.5%
B-Procedure	2891	 1.9%
I-Procedure	2819	 1.9%
B-Medication	784	 0.5%
B-Family_Member	228	 0.2%
I-Medication	116	 0.1%
I-Family_Member	9	 0.0%


#### **Configuramos pytorch y dividimos los datos.**

Importante: si tienes problemas con la ram de la gpu, disminuye el tamaño de los batches

In [ ]:
BATCH_SIZE = 16  # disminuir si hay problemas de ram.

# Usar cuda si es que está disponible.
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Using', device)

# Dividir datos entre entrenamiento y test. Si van a hacer algún sort no puede ser sobre
# el conjunto de testing ya que al hacer sus predicciones sobre el conjunto de test sin etiquetas
# debe conservar el orden original para ser comparado con los golden_labels. 

train_iterator, valid_iterator, test_iterator = legacy.data.BucketIterator.splits(
    (train_data, valid_data, test_data),
    batch_size=BATCH_SIZE,
    device=device,
    sort=False,
)

Using cuda


#### **Métricas de evaluación**

Además, definiremos las métricas que serán usadas tanto para la competencia como para evaluar el modelo: `precision`, `recall` y `micro f1-score`.
**Importante**: Noten que la evaluación solo se hace para las Named Entities (sin contar 'O'), toda esta funcionalidad nos la entrega la librería seqeval, pueden revisar más documentación aquí: https://github.com/chakki-works/seqeval. No utilicen el código entregado por sklearn para calcular las métricas ya que esta lo hace a nivel de token y no a nivel de entidad.

In [ ]:
!pip install seqeval

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
# Definimos las métricas

from seqeval.metrics import f1_score, precision_score, recall_score

def calculate_metrics(preds, y_true, pad_idx=PAD_TAG_IDX, o_idx=O_TAG_IDX):
    """
    Calcula precision, recall y f1 de cada batch.
    """

    # Obtener el indice de la clase con probabilidad mayor. (clases)
    y_pred = preds.argmax(dim=1, keepdim=True)

    # filtramos <pad> para calcular los scores.
    mask = [(y_true != pad_idx)]
    y_pred = y_pred[mask]
    y_true = y_true[mask]

    # traemos a la cpu
    y_pred = y_pred.view(-1).to('cpu').numpy()
    y_true = y_true.to('cpu').numpy()
    y_pred = [[NER_TAGS.vocab.itos[v] for v in y_pred]]
    y_true = [[NER_TAGS.vocab.itos[v] for v in y_true]]
    
    # calcular scores
    f1 = f1_score(y_true, y_pred, mode='strict')
    precision = precision_score(y_true, y_pred, mode='strict')
    recall = recall_score(y_true, y_pred, mode='strict')

    return precision, recall, f1

-------------------

### **Modelo Baseline**

Teniendo ya cargado los datos, toca definir nuestro modelo. Este baseline tendrá una capa de embedding, unas cuantas LSTM y una capa de salida y usará dropout en el entrenamiento.

Este constará de los siguientes pasos: 

1. Definir la clase que contendrá la red.
2. Definir los hiperparámetros e inicializar la red. 
3. Definir el número de épocas de entrenamiento
4. Definir la función de loss.



Recomendamos que para experimentar, encapsules los modelos en una sola variable y luego la fijes en model para entrenarla

In [ ]:
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim


# Definir la red
class NER_RNN(nn.Module):
    def __init__(self, 
                 input_dim, 
                 embedding_dim, 
                 hidden_dim, 
                 output_dim,
                 n_layers, 
                 bidirectional, 
                 dropout, 
                 pad_idx):

        super().__init__()

        # Capa de embedding
        self.embedding = nn.Embedding(input_dim,
                                      embedding_dim,
                                      padding_idx=pad_idx)

        # Capa LSTM
        self.lstm = nn.LSTM(embedding_dim,
                           hidden_dim,
                           num_layers=n_layers,
                           bidirectional=bidirectional, 
                           dropout = dropout if n_layers > 1 else 0)

        # Capa de salida
        self.fc = nn.Linear(hidden_dim * 2 if bidirectional else hidden_dim,
                            output_dim)

        # Dropout
        self.dropout = nn.Dropout(dropout)

    def forward(self, text):

        #text = [sent len, batch size]

        # Convertir lo enviado a embedding
        embedded = self.dropout(self.embedding(text))
        
        outputs, (hidden, cell) = self.lstm(embedded)
        #embedded = [sent len, batch size, emb dim]

        # Pasar los embeddings por la rnn (LSTM)

        #output = [sent len, batch size, hid dim * n directions]
        #hidden/cell = [n layers * n directions, batch size, hid dim]

        # Predecir usando la capa de salida.
        predictions = self.fc(self.dropout(outputs))
        #predictions = [sent len, batch size, output dim]

        return predictions

#### **Hiperparámetros de la red**



In [ ]:
# tamaño del vocabulario. recuerden que la entrada son vectores bag of word(one-hot).
INPUT_DIM = len(TEXT.vocab)
EMBEDDING_DIM = 300  # dimensión de los embeddings. 200
HIDDEN_DIM = 256  # dimensión de la capas LSTM 128
OUTPUT_DIM = len(NER_TAGS.vocab)  # número de clases

N_LAYERS = 3  # número de capas.
DROPOUT = 0.3  # 0.3
BIDIRECTIONAL = False

# Creamos nuestro modelo.
baseline_model = NER_RNN(INPUT_DIM, EMBEDDING_DIM, HIDDEN_DIM, OUTPUT_DIM,
                         N_LAYERS, BIDIRECTIONAL, DROPOUT, PAD_IDX)

baseline_model_name = 'baseline'  # nombre que tendrá el modelo guardado....

In [ ]:
baseline_n_epochs = 10

#### Definimos la función de loss

In [ ]:
# Loss: Cross Entropy
TAG_PAD_IDX = NER_TAGS.vocab.stoi[NER_TAGS.pad_token]
baseline_criterion = nn.CrossEntropyLoss(ignore_index = TAG_PAD_IDX)

------
## **Entrenamos y evaluamos**


**Importante** : Fijen el modelo, el número de épocas de entrenamiento, la loss y el optimizador que usarán para entrenar y evaluar en las siguientes variables!!!

In [ ]:
model = baseline_model
model_name = baseline_model_name
criterion = baseline_criterion
n_epochs = baseline_n_epochs



#### **Inicializamos la red**

Iniciamos los pesos de la red de forma aleatoria (Usando una distribución normal).


In [ ]:
def init_weights(m):
    # Inicializamos los pesos como aleatorios
    for name, param in m.named_parameters():
        nn.init.normal_(param.data, mean=0, std=0.1) 
        
    # Seteamos como 0 los embeddings de UNK y PAD
    model.embedding.weight.data[UNK_IDX] = torch.zeros(EMBEDDING_DIM)
    model.embedding.weight.data[PAD_IDX] = torch.zeros(EMBEDDING_DIM)
        
model.apply(init_weights)

NER_RNN(
  (embedding): Embedding(17591, 300, padding_idx=1)
  (lstm): LSTM(300, 256, num_layers=3, dropout=0.3)
  (fc): Linear(in_features=256, out_features=12, bias=True)
  (dropout): Dropout(p=0.3, inplace=False)
)

In [ ]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)
print(f'El modelo actual tiene {count_parameters(model):,} parámetros entrenables.')

El modelo actual tiene 6,904,448 parámetros entrenables.


Notar que definimos los embeddings que representan a \<unk\> y \<pad\>  como [0, 0, ..., 0]

#### **Definimos el optimizador**

In [ ]:
# Optimizador
optimizer = optim.Adam(model.parameters())

#### **Enviamos el modelo a cuda**


In [ ]:
# Enviamos el modelo y la loss a cuda (en el caso en que esté disponible)
model = model.to(device)
criterion = criterion.to(device)

#### **Definimos el entrenamiento de la red**

Algunos conceptos previos: 

- `epoch` : una pasada de entrenamiento completa de una dataset.
- `batch`: una fracción de la época. Se utilizan para entrenar mas rápidamente la red. (mas eficiente pasar n datos que uno en cada ejecución del backpropagation)

Esta función está encargada de entrenar la red en una época. Para esto, por cada batch de la época actual, predice los tags del texto, calcula su loss y luego hace backpropagation para actualizar los pesos de la red.

Observación: En algunos comentarios aparecerá el tamaño de los tensores entre corchetes

In [ ]:
def train(model, iterator, optimizer, criterion):

    epoch_loss = 0
    epoch_precision = 0
    epoch_recall = 0
    epoch_f1 = 0

    model.train()

    # Por cada batch del iterador de la época:
    for batch in iterator:

        # Extraemos el texto y los tags del batch que estamos procesado
        text = batch.text
        tags = batch.nertags

        # Reiniciamos los gradientes calculados en la iteración anterior
        optimizer.zero_grad()

        #text = [sent len, batch size]

        # Predecimos los tags del texto del batch.
        predictions = model(text)

        #predictions = [sent len, batch size, output dim]
        #tags = [sent len, batch size]

        # Reordenamos los datos para calcular la loss
        predictions = predictions.view(-1, predictions.shape[-1])
        tags = tags.view(-1)

        #predictions = [sent len * batch size, output dim]



        # Calculamos el Cross Entropy de las predicciones con respecto a las etiquetas reales
        loss = criterion(predictions, tags)
        
        # Calculamos el accuracy
        precision, recall, f1 = calculate_metrics(predictions, tags)

        # Calculamos los gradientes
        loss.backward()

        # Actualizamos los parámetros de la red
        optimizer.step()

        # Actualizamos el loss y las métricas
        epoch_loss += loss.item()
        epoch_precision += precision
        epoch_recall += recall
        epoch_f1 += f1

    return epoch_loss / len(iterator), epoch_precision / len(
        iterator), epoch_recall / len(iterator), epoch_f1 / len(iterator)

#### **Definimos la función de evaluación**

Evalua el rendimiento actual de la red usando los datos de validación. 

Por cada batch de estos datos, calcula y reporta el loss y las métricas asociadas al conjunto de validación. 
Ya que las métricas son calculadas por cada batch, estas son retornadas promediadas por el número de batches entregados. (ver linea del return)

In [ ]:
def evaluate(model, iterator, criterion):

    epoch_loss = 0
    epoch_precision = 0
    epoch_recall = 0
    epoch_f1 = 0

    model.eval()

    # Indicamos que ahora no guardaremos los gradientes
    with torch.no_grad():
        # Por cada batch
        for batch in iterator:

            text = batch.text
            tags = batch.nertags

            # Predecimos
            predictions = model(text)

            predictions = predictions.view(-1, predictions.shape[-1])
            tags = tags.view(-1)

            # Calculamos el Cross Entropy de las predicciones con respecto a las etiquetas reales
            loss = criterion(predictions, tags)

            # Calculamos las métricas
            precision, recall, f1 = calculate_metrics(predictions, tags)

            # Actualizamos el loss y las métricas
            epoch_loss += loss.item()
            epoch_precision += precision
            epoch_recall += recall
            epoch_f1 += f1

    return epoch_loss / len(iterator), epoch_precision / len(
        iterator), epoch_recall / len(iterator), epoch_f1 / len(iterator)

In [ ]:
import time

def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs


#### **Entrenamiento de la red**

En este cuadro de código ejecutaremos el entrenamiento de la red.
Para esto, primero definiremos el número de épocas y luego por cada época, ejecutaremos `train` y `evaluate`.

**Importante: Reiniciar los pesos del modelo**

Si ejecutas nuevamente esta celda, se seguira entrenando el mismo modelo una y otra vez. 
Para reiniciar el modelo se debe ejecutar nuevamente la celda que contiene la función `init_weights`



In [ ]:
best_valid_loss = float('inf')

for epoch in range(n_epochs):

    start_time = time.time()

    # Recuerdo: train_iterator y valid_iterator contienen el dataset dividido en batches.

    # Entrenar
    train_loss, train_precision, train_recall, train_f1 = train(
        model, train_iterator, optimizer, criterion)

    # Evaluar (valid = validación)
    valid_loss, valid_precision, valid_recall, valid_f1 = evaluate(
        model, valid_iterator, criterion)

    end_time = time.time()

    epoch_mins, epoch_secs = epoch_time(start_time, end_time)

    # Si obtuvimos mejores resultados, guardamos este modelo en el almacenamiento (para poder cargarlo luego)
    # Si detienen el entrenamiento prematuramente, pueden cargar el modelo en el siguiente recuadro de código.
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), '{}.pt'.format(model_name))
    # Si ya no mejoramos el loss de validación, terminamos de entrenar.

    print(f'Epoch: {epoch+1:02} | Epoch Time: {epoch_mins}m {epoch_secs}s')
    print(
        f'\tTrain Loss: {train_loss:.3f} | Train f1: {train_f1:.2f} | Train precision: {train_precision:.2f} | Train recall: {train_recall:.2f}'
    )
    print(
        f'\t Val. Loss: {valid_loss:.3f} |  Val. f1: {valid_f1:.2f} |  Val. precision: {valid_precision:.2f} | Val. recall: {valid_recall:.2f}'
    )

/usr/local/lib/python3.7/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: <pad> seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.7/dist-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Epoch: 01 | Epoch Time: 0m 10s
	Train Loss: 0.748 | Train f1: 0.44 | Train precision: 0.59 | Train recall: 0.37
	 Val. Loss: 0.469 |  Val. f1: 0.65 |  Val. precision: 0.74 | Val. recall: 0.58
Epoch: 02 | Epoch Time: 0m 8s
	Train Loss: 0.368 | Train f1: 0.74 | Train precision: 0.77 | Train recall: 0.71
	 Val. Loss: 0.406 |  Val. f1: 0.70 |  Val. precision: 0.71 | Val. recall: 0.70
Epoch: 03 | Epoch Time: 0m 8s
	Train Loss: 0.246 | Train f1: 0.82 | Train precision: 0.83 | Train recall: 0.81
	 Val. Loss: 0.381 |  Val. f1: 0.74 |  Val. precision: 0.76 | Val. recall: 0.73
Epoch: 04 | Epoch Time: 0m 8s
	Train Loss: 0.185 | Train f1: 0.86 | Train precision: 0.87 | Train recall: 0.86
	 Val. Loss: 0.388 |  Val. f1: 0.74 |  Val. precision: 0.77 | Val. recall: 0.71
Epoch: 05 | Epoch Time: 0m 8s
	Train Loss: 0.149 | Train f1: 0.89 | Train precision: 0.89 | Train recall: 0.89
	 Val. Loss: 0.431 |  Val. f1: 0.72 |  Val. precision: 0.70 | Val. recall: 0.75
Epoch: 06 | Epoch Time: 0m 8s
	Train Loss: 0

**Importante**: Recuerden que el último modelo entrenado no es el mejor (probablemente esté *overfitteado*), si no el que guardamos con la menor loss del conjunto de validación. Este problema lo pueden solucionar con *early stopping*.
Para cargar el mejor modelo entrenado, ejecuten la siguiente celda.



In [ ]:
# cargar el mejor modelo entrenado.
model.load_state_dict(torch.load('{}.pt'.format(model_name)))

<All keys matched successfully>

In [ ]:
# Limpiar ram de cuda
torch.cuda.empty_cache()

#### **Evaluamos el set de validación con el modelo final**

Estos son los resultados de predecir el dataset de evaluación con el *mejor* modelo entrenado.

In [ ]:
valid_loss, valid_precision, valid_recall, valid_f1 = evaluate(
    model, valid_iterator, criterion)

print(
    f'Val. Loss: {valid_loss:.3f} |  Val. f1: {valid_f1:.2f} | Val. precision: {valid_precision:.2f} | Val. recall: {valid_recall:.2f}'
)

Val. Loss: 0.381 |  Val. f1: 0.74 | Val. precision: 0.76 | Val. recall: 0.73


###Experimentacion EarlyStopping
La primera experimentacion que se propone es la utilizacion de earlyStopping para normalizar y evitar el overfitting ademas de proveer la facilidad de no tener que utilizar tiempo en exceso al experimentar con las cantidad de epocas.

In [ ]:
#Los argumetnos de la funciones son el modelo fijado, y el argumento stop indica cuando debe terminar el entrenamiento cuando no da mejor loss
def earlyStop_model(model, train_iterator, valid_iterator, optimizer, 
                   criterion, stop = 10):

  #nro de epocas
  Epochs = 100
  counter = 0
  best_epoch = 0
  best_valid_loss = float('inf')
  last_valid_loss = float('inf')
  
  for epoch in range(Epochs):
    
      start_time = time.time()

      # Recuerdo: train_iterator y valid_iterator contienen el dataset dividido en batches.

      # Entrenar
      train_loss, train_precision, train_recall, train_f1 = train(
          model, train_iterator, optimizer, criterion)

      # Evaluar (valid = validación)
      valid_loss, valid_precision, valid_recall, valid_f1 = evaluate(
          model, valid_iterator, criterion)

      end_time = time.time()

      epoch_mins, epoch_secs = epoch_time(start_time, end_time)

      print(f'Epoch: {epoch+1:02} | Epoch Time: {epoch_mins}m {epoch_secs}s')
      print(
          f'\tTrain Loss: {train_loss:.3f} | Train f1: {train_f1:.2f} | Train precision: {train_precision:.2f} | Train recall: {train_recall:.2f}'
      )
      print(
          f'\t Val. Loss: {valid_loss:.3f} |  Val. f1: {valid_f1:.2f} |  Val. precision: {valid_precision:.2f} | Val. recall: {valid_recall:.2f}'
      )

      # Si obtuvimos mejores resultados, guardamos este modelo en el almacenamiento (para poder cargarlo luego)
      # Si detienen el entrenamiento prematuramente, pueden cargar el modelo en el siguiente recuadro de código.
      if valid_loss < best_valid_loss:
          best_valid_loss = valid_loss
          torch.save(model.state_dict(), '{}.pt'.format(model_name))
          best_epoch = epoch
          counter = 0

      # Si ya no mejoramos el loss de validación, terminamos de entrenar.
      else:
          counter += 1
                   
          if counter == stop:
              break

      last_valid_loss = valid_loss

  # cargar el mejor modelo entrenado.
  model.load_state_dict(torch.load('{}.pt'.format(model_name)))
  # Limpiar ram de cuda
  torch.cuda.empty_cache()

  valid_loss, valid_precision, valid_recall, valid_f1 = evaluate(
    model, valid_iterator, criterion)
  
  print(f'\nBest Model:')
  print(
      f'Val. Loss: {valid_loss:.3f} |  Val. f1: {valid_f1:.2f} | Val. precision: {valid_precision:.2f} | Val. recall: {valid_recall:.2f}'
  )

###Ejemplo de uso con el modelo Baseline

In [ ]:
model.apply(init_weights)
earlyStop_model(model, train_iterator, valid_iterator, optimizer, criterion)

/usr/local/lib/python3.7/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: <pad> seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.7/dist-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Epoch: 01 | Epoch Time: 0m 8s
	Train Loss: 1.028 | Train f1: 0.17 | Train precision: 0.44 | Train recall: 0.11
	 Val. Loss: 0.718 |  Val. f1: 0.45 |  Val. precision: 0.70 | Val. recall: 0.34
Epoch: 02 | Epoch Time: 0m 10s
	Train Loss: 0.512 | Train f1: 0.64 | Train precision: 0.73 | Train recall: 0.57
	 Val. Loss: 0.440 |  Val. f1: 0.67 |  Val. precision: 0.77 | Val. recall: 0.61
Epoch: 03 | Epoch Time: 0m 8s
	Train Loss: 0.298 | Train f1: 0.78 | Train precision: 0.80 | Train recall: 0.77
	 Val. Loss: 0.381 |  Val. f1: 0.73 |  Val. precision: 0.73 | Val. recall: 0.72
Epoch: 04 | Epoch Time: 0m 8s
	Train Loss: 0.212 | Train f1: 0.84 | Train precision: 0.85 | Train recall: 0.84
	 Val. Loss: 0.383 |  Val. f1: 0.74 |  Val. precision: 0.75 | Val. recall: 0.74
Epoch: 05 | Epoch Time: 0m 8s
	Train Loss: 0.164 | Train f1: 0.88 | Train precision: 0.88 | Train recall: 0.88
	 Val. Loss: 0.412 |  Val. f1: 0.74 |  Val. precision: 0.75 | Val. recall: 0.73
Epoch: 06 | Epoch Time: 0m 8s
	Train Loss: 0

##Experimentacion modelos Elman y GRU
Como alternativa a las redes recurentes LSTM, se procede a experimentar con modelos GRU y Elmann unidireccionales ya que estas ya se encuentran implementadas en Pytorch, ademas de utilizar la funcion earlyStop_model para evitar el overfitting.

In [ ]:
class ELMAN(nn.Module):
    def __init__(self, 
                 input_dim, 
                 embedding_dim, 
                 hidden_dim, 
                 output_dim,
                 n_layers, 
                 bidirectional, 
                 dropout, 
                 pad_idx,
                 nonlinearity = 'relu'):     #Por defecto se usa tanh pero para experimenter la cambie a relu

        super().__init__()

        # Capa de embedding
        self.embedding = nn.Embedding(input_dim,
                                      embedding_dim,
                                      padding_idx=pad_idx)

        # Capa Elman RNN
        self.rnn = nn.RNN(embedding_dim,
                           hidden_dim,
                           num_layers=n_layers,
                           bidirectional=bidirectional, 
                           dropout = dropout if n_layers > 1 else 0,
                           nonlinearity = nonlinearity)

        # Capa de salida
        self.fc = nn.Linear(hidden_dim * 2 if bidirectional else hidden_dim,
                            output_dim)

        # Dropout
        self.dropout = nn.Dropout(dropout)

    def forward(self, text):

        #text = [sent len, batch size]

        # Convertir lo enviado a embedding
        embedded = self.dropout(self.embedding(text))
        
        outputs, hidden = self.rnn(embedded)
        #embedded = [sent len, batch size, emb dim]

        # Pasar los embeddings por la rnn (LSTM)

        #output = [sent len, batch size, hid dim * n directions]
        #hidden/cell = [n layers * n directions, batch size, hid dim]

        # Predecir usando la capa de salida.
        predictions = self.fc(self.dropout(outputs))
        #predictions = [sent len, batch size, output dim]

        return predictions

class GRU(nn.Module):
    def __init__(self, 
                 input_dim, 
                 embedding_dim, 
                 hidden_dim, 
                 output_dim,
                 n_layers, 
                 bidirectional, 
                 dropout, 
                 pad_idx):

        super().__init__()

        # Capa de embedding
        self.embedding = nn.Embedding(input_dim,
                                      embedding_dim,
                                      padding_idx=pad_idx)

        # Capa GRU RNN
        self.gru = nn.GRU(embedding_dim,
                           hidden_dim,
                           num_layers=n_layers,
                           bidirectional=bidirectional, 
                           dropout = dropout if n_layers > 1 else 0)

        # Capa de salida
        self.fc = nn.Linear(hidden_dim * 2 if bidirectional else hidden_dim,
                            output_dim)

        # Dropout
        self.dropout = nn.Dropout(dropout)

    def forward(self, text):

        #text = [sent len, batch size]

        # Convertir lo enviado a embedding
        embedded = self.dropout(self.embedding(text))
        
        outputs, hidden = self.gru(embedded)
        #embedded = [sent len, batch size, emb dim]

        # Pasar los embeddings por la rnn (LSTM)

        #output = [sent len, batch size, hid dim * n directions]
        #hidden/cell = [n layers * n directions, batch size, hid dim]

        # Predecir usando la capa de salida.
        predictions = self.fc(self.dropout(outputs))
        #predictions = [sent len, batch size, output dim]

        return predictions



###Hiperparametros Elman y GRU

In [ ]:
# tamaño del vocabulario. recuerden que la entrada son vectores bag of word(one-hot).
INPUT_DIM = len(TEXT.vocab)
EMBEDDING_DIM = 300  # dimensión de los embeddings. 
HIDDEN_DIM = 256  # dimensión de la capas LSTM 
OUTPUT_DIM = len(NER_TAGS.vocab)  # número de clases

N_LAYERS = 3  # número de capas.
DROPOUT = 0.3 
BIDIRECTIONAL = False


# Creamos nuestros modelos.
elman_model = ELMAN(INPUT_DIM, EMBEDDING_DIM, HIDDEN_DIM, OUTPUT_DIM,
                         N_LAYERS, BIDIRECTIONAL, DROPOUT, PAD_IDX)

elman_model_name = 'Elman'  # nombre que tendrá el modelo guardado...

gru_model = GRU(INPUT_DIM, EMBEDDING_DIM, HIDDEN_DIM, OUTPUT_DIM,
                         N_LAYERS, BIDIRECTIONAL, DROPOUT, PAD_IDX)

gru_model_name = 'GRU'  # nombre que tendrá el modelo guardado...

###Modelo Elman

In [ ]:
model = elman_model
model_name = baseline_model_name
criterion = baseline_criterion

model.apply(init_weights)
print(f'El modelo actual tiene {count_parameters(model):,} parámetros entrenables.')

optimizer = optim.Adam(model.parameters())
model = model.to(device)
criterion = criterion.to(device)

earlyStop_model(model, train_iterator, valid_iterator, optimizer, criterion)

El modelo actual tiene 5,686,400 parámetros entrenables.


/usr/local/lib/python3.7/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: <pad> seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.7/dist-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Epoch: 01 | Epoch Time: 0m 7s
	Train Loss: 2.718 | Train f1: 0.05 | Train precision: 0.12 | Train recall: 0.04
	 Val. Loss: 1.181 |  Val. f1: 0.00 |  Val. precision: 0.04 | Val. recall: 0.00
Epoch: 02 | Epoch Time: 0m 6s
	Train Loss: 1.130 | Train f1: 0.05 | Train precision: 0.33 | Train recall: 0.03
	 Val. Loss: 1.125 |  Val. f1: 0.11 |  Val. precision: 0.50 | Val. recall: 0.06
Epoch: 03 | Epoch Time: 0m 6s
	Train Loss: 1.060 | Train f1: 0.11 | Train precision: 0.45 | Train recall: 0.07
	 Val. Loss: 1.017 |  Val. f1: 0.21 |  Val. precision: 0.54 | Val. recall: 0.13
Epoch: 04 | Epoch Time: 0m 7s
	Train Loss: 0.954 | Train f1: 0.23 | Train precision: 0.53 | Train recall: 0.16
	 Val. Loss: 0.886 |  Val. f1: 0.35 |  Val. precision: 0.60 | Val. recall: 0.25
Epoch: 05 | Epoch Time: 0m 6s
	Train Loss: 0.806 | Train f1: 0.38 | Train precision: 0.60 | Train recall: 0.28
	 Val. Loss: 0.716 |  Val. f1: 0.45 |  Val. precision: 0.66 | Val. recall: 0.35
Epoch: 06 | Epoch Time: 0m 6s
	Train Loss: 0.

###Modelo GRU

In [ ]:
model = gru_model
model_name = baseline_model_name
criterion = baseline_criterion

model.apply(init_weights)
print(f'El modelo actual tiene {count_parameters(model):,} parámetros entrenables.')

optimizer = optim.Adam(model.parameters())
model = model.to(device)
criterion = criterion.to(device)

earlyStop_model(model, train_iterator, valid_iterator, optimizer, criterion)

El modelo actual tiene 6,498,432 parámetros entrenables.


/usr/local/lib/python3.7/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: <pad> seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))


Epoch: 01 | Epoch Time: 0m 8s
	Train Loss: 0.756 | Train f1: 0.46 | Train precision: 0.59 | Train recall: 0.40
	 Val. Loss: 0.450 |  Val. f1: 0.67 |  Val. precision: 0.75 | Val. recall: 0.61
Epoch: 02 | Epoch Time: 0m 8s
	Train Loss: 0.370 | Train f1: 0.73 | Train precision: 0.76 | Train recall: 0.70
	 Val. Loss: 0.399 |  Val. f1: 0.72 |  Val. precision: 0.78 | Val. recall: 0.67
Epoch: 03 | Epoch Time: 0m 8s
	Train Loss: 0.244 | Train f1: 0.82 | Train precision: 0.83 | Train recall: 0.81
	 Val. Loss: 0.409 |  Val. f1: 0.74 |  Val. precision: 0.75 | Val. recall: 0.74
Epoch: 04 | Epoch Time: 0m 8s
	Train Loss: 0.185 | Train f1: 0.86 | Train precision: 0.87 | Train recall: 0.86
	 Val. Loss: 0.419 |  Val. f1: 0.74 |  Val. precision: 0.75 | Val. recall: 0.73
Epoch: 05 | Epoch Time: 0m 8s
	Train Loss: 0.148 | Train f1: 0.89 | Train precision: 0.89 | Train recall: 0.89
	 Val. Loss: 0.452 |  Val. f1: 0.74 |  Val. precision: 0.75 | Val. recall: 0.73
Epoch: 06 | Epoch Time: 0m 8s
	Train Loss: 0.

##Experimentacion Bidireccionlidad
Para esta seccion se estudiara el efecto de la bidereccionalidad de los modelos LSTM, GRU y Elmann.

In [ ]:
# tamaño del vocabulario. recuerden que la entrada son vectores bag of word(one-hot).
INPUT_DIM = len(TEXT.vocab)
EMBEDDING_DIM = 300  # dimensión de los embeddings. 
HIDDEN_DIM = 256  # dimensión de la capas LSTM 
OUTPUT_DIM = len(NER_TAGS.vocab)  # número de clases

N_LAYERS = 3  # número de capas.
DROPOUT = 0.3  
BIDIRECTIONAL = True

# Creamos nuestros modelos.
lstm_model = NER_RNN(INPUT_DIM, EMBEDDING_DIM, HIDDEN_DIM, OUTPUT_DIM,
                         N_LAYERS, BIDIRECTIONAL, DROPOUT, PAD_IDX)

lstm_model_name = 'LSTMbid'  # nombre que tendrá el modelo guardado...

elman_model = ELMAN(INPUT_DIM, EMBEDDING_DIM, HIDDEN_DIM, OUTPUT_DIM,
                         N_LAYERS, BIDIRECTIONAL, DROPOUT, PAD_IDX, 'relu')

elman_model_name = 'Elmanbid'  # nombre que tendrá el modelo guardado...

gru_model = GRU(INPUT_DIM, EMBEDDING_DIM, HIDDEN_DIM, OUTPUT_DIM,
                         N_LAYERS, BIDIRECTIONAL, DROPOUT, PAD_IDX)

gru_model_name = 'GRUbid'  # nombre que tendrá el modelo guardado...

####LSTM-bidireccional

In [ ]:
model = lstm_model
model_name = lstm_model_name
criterion = baseline_criterion

#inicializamos los pesos
model.apply(init_weights)
print(f'El modelo actual tiene {count_parameters(model):,} parámetros entrenables.')

#se determina el optimizador
optimizer = optim.Adam(model.parameters())
model = model.to(device)
criterion = criterion.to(device)

earlyStop_model(model, train_iterator, valid_iterator, optimizer, criterion)

El modelo actual tiene 15,818,732 parámetros entrenables.


/usr/local/lib/python3.7/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: <pad> seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.7/dist-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Epoch: 01 | Epoch Time: 0m 13s
	Train Loss: 0.634 | Train f1: 0.55 | Train precision: 0.66 | Train recall: 0.49
	 Val. Loss: 0.397 |  Val. f1: 0.71 |  Val. precision: 0.78 | Val. recall: 0.65
Epoch: 02 | Epoch Time: 0m 13s
	Train Loss: 0.281 | Train f1: 0.80 | Train precision: 0.82 | Train recall: 0.78
	 Val. Loss: 0.355 |  Val. f1: 0.76 |  Val. precision: 0.78 | Val. recall: 0.74
Epoch: 03 | Epoch Time: 0m 13s
	Train Loss: 0.158 | Train f1: 0.89 | Train precision: 0.89 | Train recall: 0.89
	 Val. Loss: 0.374 |  Val. f1: 0.77 |  Val. precision: 0.78 | Val. recall: 0.76
Epoch: 04 | Epoch Time: 0m 13s
	Train Loss: 0.106 | Train f1: 0.92 | Train precision: 0.92 | Train recall: 0.92
	 Val. Loss: 0.404 |  Val. f1: 0.76 |  Val. precision: 0.78 | Val. recall: 0.75
Epoch: 05 | Epoch Time: 0m 13s
	Train Loss: 0.078 | Train f1: 0.94 | Train precision: 0.94 | Train recall: 0.94
	 Val. Loss: 0.419 |  Val. f1: 0.76 |  Val. precision: 0.75 | Val. recall: 0.77
Epoch: 06 | Epoch Time: 0m 13s
	Train Lo

####Elman-bidireccional

In [ ]:
model = elman_model
model_name = elman_model_name
criterion = baseline_criterion
#inicializar pesos
model.apply(init_weights)
print(f'El modelo actual tiene {count_parameters(model):,} parámetros entrenables.')
#optimizador
optimizer = optim.Adam(model.parameters())
model = model.to(device)
criterion = criterion.to(device)

earlyStop_model(model, train_iterator, valid_iterator, optimizer, criterion)

El modelo actual tiene 6,357,632 parámetros entrenables.


/usr/local/lib/python3.7/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: <pad> seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))


Epoch: 01 | Epoch Time: 0m 10s
	Train Loss: nan | Train f1: 0.04 | Train precision: 0.06 | Train recall: 0.04
	 Val. Loss: nan |  Val. f1: 0.00 |  Val. precision: 0.00 | Val. recall: 0.00
Epoch: 02 | Epoch Time: 0m 10s
	Train Loss: nan | Train f1: 0.00 | Train precision: 0.00 | Train recall: 0.00
	 Val. Loss: nan |  Val. f1: 0.00 |  Val. precision: 0.00 | Val. recall: 0.00
Epoch: 03 | Epoch Time: 0m 10s
	Train Loss: nan | Train f1: 0.00 | Train precision: 0.00 | Train recall: 0.00
	 Val. Loss: nan |  Val. f1: 0.00 |  Val. precision: 0.00 | Val. recall: 0.00
Epoch: 04 | Epoch Time: 0m 10s
	Train Loss: nan | Train f1: 0.00 | Train precision: 0.00 | Train recall: 0.00
	 Val. Loss: nan |  Val. f1: 0.00 |  Val. precision: 0.00 | Val. recall: 0.00
Epoch: 05 | Epoch Time: 0m 10s
	Train Loss: nan | Train f1: 0.00 | Train precision: 0.00 | Train recall: 0.00
	 Val. Loss: nan |  Val. f1: 0.00 |  Val. precision: 0.00 | Val. recall: 0.00
Epoch: 06 | Epoch Time: 0m 10s
	Train Loss: nan | Train f1: 

####GRU-bidireccional

In [ ]:
model = gru_model
model_name = gru_model_name
criterion = baseline_criterion
#inicializar pesos
model.apply(init_weights)
print(f'El modelo actual tiene {count_parameters(model):,} parámetros entrenables.')
#optimizador
optimizer = optim.Adam(model.parameters())
model = model.to(device)
criterion = criterion.to(device)

earlyStop_model(model, train_iterator, valid_iterator, optimizer, criterion)

El modelo actual tiene 8,505,984 parámetros entrenables.


/usr/local/lib/python3.7/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: <pad> seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.7/dist-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Epoch: 01 | Epoch Time: 0m 12s
	Train Loss: 0.792 | Train f1: 0.48 | Train precision: 0.57 | Train recall: 0.43
	 Val. Loss: 0.425 |  Val. f1: 0.69 |  Val. precision: 0.77 | Val. recall: 0.63
Epoch: 02 | Epoch Time: 0m 12s
	Train Loss: 0.357 | Train f1: 0.75 | Train precision: 0.78 | Train recall: 0.72
	 Val. Loss: 0.366 |  Val. f1: 0.75 |  Val. precision: 0.78 | Val. recall: 0.73
Epoch: 03 | Epoch Time: 0m 12s
	Train Loss: 0.216 | Train f1: 0.84 | Train precision: 0.85 | Train recall: 0.83
	 Val. Loss: 0.376 |  Val. f1: 0.75 |  Val. precision: 0.77 | Val. recall: 0.75
Epoch: 04 | Epoch Time: 0m 12s
	Train Loss: 0.148 | Train f1: 0.89 | Train precision: 0.89 | Train recall: 0.88
	 Val. Loss: 0.404 |  Val. f1: 0.77 |  Val. precision: 0.80 | Val. recall: 0.74
Epoch: 05 | Epoch Time: 0m 12s
	Train Loss: 0.108 | Train f1: 0.92 | Train precision: 0.92 | Train recall: 0.92
	 Val. Loss: 0.426 |  Val. f1: 0.77 |  Val. precision: 0.78 | Val. recall: 0.76
Epoch: 06 | Epoch Time: 0m 12s
	Train Lo

##Experimentacion arquitecturas
Para esta seccion se estudiara el efecto de distintas arquitecturas sobre las bidireccionales

####LSTM - bi - variando embedding

In [ ]:
INPUT_DIM = len(TEXT.vocab)
EMBEDDING_DIM = 500  # dimensión de los embeddings. 
HIDDEN_DIM = 256  # dimensión de la capas LSTM 
OUTPUT_DIM = len(NER_TAGS.vocab)  # número de clases

N_LAYERS = 3  # número de capas.
DROPOUT = 0.3  
BIDIRECTIONAL = True

# Creamos nuestros modelos.
lstm_model = NER_RNN(INPUT_DIM, EMBEDDING_DIM, HIDDEN_DIM, OUTPUT_DIM,
                         N_LAYERS, BIDIRECTIONAL, DROPOUT, PAD_IDX)

lstm_model_name = 'LSTMbidEMD'  # nombre que tendrá el modelo guardado...

In [ ]:
model = lstm_model
model_name = lstm_model_name
criterion = baseline_criterion

#inicializamos los pesos
model.apply(init_weights)
print(f'El modelo actual tiene {count_parameters(model):,} parámetros entrenables.')

#se determina el optimizador
optimizer = optim.Adam(model.parameters())
model = model.to(device)
criterion = criterion.to(device)

earlyStop_model(model, train_iterator, valid_iterator, optimizer, criterion)

El modelo actual tiene 13,507,960 parámetros entrenables.


/usr/local/lib/python3.7/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: <pad> seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.7/dist-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Epoch: 01 | Epoch Time: 0m 14s
	Train Loss: 0.646 | Train f1: 0.54 | Train precision: 0.66 | Train recall: 0.47
	 Val. Loss: 0.390 |  Val. f1: 0.70 |  Val. precision: 0.78 | Val. recall: 0.64
Epoch: 02 | Epoch Time: 0m 14s
	Train Loss: 0.287 | Train f1: 0.79 | Train precision: 0.82 | Train recall: 0.77
	 Val. Loss: 0.340 |  Val. f1: 0.76 |  Val. precision: 0.78 | Val. recall: 0.75
Epoch: 03 | Epoch Time: 0m 14s
	Train Loss: 0.170 | Train f1: 0.87 | Train precision: 0.88 | Train recall: 0.87
	 Val. Loss: 0.353 |  Val. f1: 0.76 |  Val. precision: 0.76 | Val. recall: 0.77
Epoch: 04 | Epoch Time: 0m 14s
	Train Loss: 0.115 | Train f1: 0.91 | Train precision: 0.92 | Train recall: 0.91
	 Val. Loss: 0.395 |  Val. f1: 0.77 |  Val. precision: 0.77 | Val. recall: 0.76
Epoch: 05 | Epoch Time: 0m 14s
	Train Loss: 0.079 | Train f1: 0.94 | Train precision: 0.94 | Train recall: 0.94
	 Val. Loss: 0.439 |  Val. f1: 0.76 |  Val. precision: 0.77 | Val. recall: 0.75
Epoch: 06 | Epoch Time: 0m 14s
	Train Lo

####LSTM - bi - variando dimensión LSTMs = 512

In [ ]:
INPUT_DIM = len(TEXT.vocab)
EMBEDDING_DIM = 300  # dimensión de los embeddings. 
HIDDEN_DIM = 512  # dimensión de la capas LSTM 
OUTPUT_DIM = len(NER_TAGS.vocab)  # número de clases

N_LAYERS = 3  # número de capas.
DROPOUT = 0.3  
BIDIRECTIONAL = True

# Creamos nuestros modelos.
lstm_model = NER_RNN(INPUT_DIM, EMBEDDING_DIM, HIDDEN_DIM, OUTPUT_DIM,
                         N_LAYERS, BIDIRECTIONAL, DROPOUT, PAD_IDX)

lstm_model_name = 'LSTMbidHID'  # nombre que tendrá el modelo guardado...

In [ ]:
model = lstm_model
model_name = lstm_model_name
criterion = baseline_criterion

#inicializamos los pesos
model.apply(init_weights)
print(f'El modelo actual tiene {count_parameters(model):,} parámetros entrenables.')

#se determina el optimizador
optimizer = optim.Adam(model.parameters())
model = model.to(device)
criterion = criterion.to(device)

earlyStop_model(model, train_iterator, valid_iterator, optimizer, criterion)

El modelo actual tiene 21,223,040 parámetros entrenables.


/usr/local/lib/python3.7/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: <pad> seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.7/dist-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Epoch: 01 | Epoch Time: 0m 28s
	Train Loss: 0.843 | Train f1: 0.37 | Train precision: 0.56 | Train recall: 0.31
	 Val. Loss: 0.500 |  Val. f1: 0.62 |  Val. precision: 0.74 | Val. recall: 0.54
Epoch: 02 | Epoch Time: 0m 28s
	Train Loss: 0.371 | Train f1: 0.74 | Train precision: 0.78 | Train recall: 0.70
	 Val. Loss: 0.368 |  Val. f1: 0.74 |  Val. precision: 0.78 | Val. recall: 0.70
Epoch: 03 | Epoch Time: 0m 29s
	Train Loss: 0.222 | Train f1: 0.84 | Train precision: 0.85 | Train recall: 0.83
	 Val. Loss: 0.368 |  Val. f1: 0.76 |  Val. precision: 0.79 | Val. recall: 0.74
Epoch: 04 | Epoch Time: 0m 28s
	Train Loss: 0.151 | Train f1: 0.89 | Train precision: 0.89 | Train recall: 0.89
	 Val. Loss: 0.386 |  Val. f1: 0.75 |  Val. precision: 0.75 | Val. recall: 0.76
Epoch: 05 | Epoch Time: 0m 29s
	Train Loss: 0.112 | Train f1: 0.92 | Train precision: 0.92 | Train recall: 0.92
	 Val. Loss: 0.405 |  Val. f1: 0.77 |  Val. precision: 0.79 | Val. recall: 0.75
Epoch: 06 | Epoch Time: 0m 28s
	Train Lo

####LSTM - bi - variando dimensión LSTMs = 128

In [ ]:
INPUT_DIM = len(TEXT.vocab)
EMBEDDING_DIM = 300  # dimensión de los embeddings. 
HIDDEN_DIM = 128  # dimensión de la capas LSTM 
OUTPUT_DIM = len(NER_TAGS.vocab)  # número de clases

N_LAYERS = 3  # número de capas.
DROPOUT = 0.3  
BIDIRECTIONAL = True

# Creamos nuestros modelos.
lstm_model = NER_RNN(INPUT_DIM, EMBEDDING_DIM, HIDDEN_DIM, OUTPUT_DIM,
                         N_LAYERS, BIDIRECTIONAL, DROPOUT, PAD_IDX)

lstm_model_name = 'LSTMbidDIM'  # nombre que tendrá el modelo guardado...

In [ ]:
model = lstm_model
model_name = lstm_model_name
criterion = baseline_criterion

#inicializamos los pesos
model.apply(init_weights)
print(f'El modelo actual tiene {count_parameters(model):,} parámetros entrenables.')

#se determina el optimizador
optimizer = optim.Adam(model.parameters())
model = model.to(device)
criterion = criterion.to(device)

earlyStop_model(model, train_iterator, valid_iterator, optimizer, criterion)

El modelo actual tiene 6,511,232 parámetros entrenables.


/usr/local/lib/python3.7/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: <pad> seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.7/dist-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Epoch: 01 | Epoch Time: 0m 11s
	Train Loss: 0.694 | Train f1: 0.50 | Train precision: 0.63 | Train recall: 0.43
	 Val. Loss: 0.420 |  Val. f1: 0.69 |  Val. precision: 0.78 | Val. recall: 0.62
Epoch: 02 | Epoch Time: 0m 11s
	Train Loss: 0.322 | Train f1: 0.76 | Train precision: 0.80 | Train recall: 0.74
	 Val. Loss: 0.357 |  Val. f1: 0.74 |  Val. precision: 0.76 | Val. recall: 0.73
Epoch: 03 | Epoch Time: 0m 11s
	Train Loss: 0.198 | Train f1: 0.86 | Train precision: 0.86 | Train recall: 0.85
	 Val. Loss: 0.347 |  Val. f1: 0.76 |  Val. precision: 0.79 | Val. recall: 0.74
Epoch: 04 | Epoch Time: 0m 11s
	Train Loss: 0.137 | Train f1: 0.90 | Train precision: 0.90 | Train recall: 0.90
	 Val. Loss: 0.374 |  Val. f1: 0.76 |  Val. precision: 0.79 | Val. recall: 0.75
Epoch: 05 | Epoch Time: 0m 11s
	Train Loss: 0.104 | Train f1: 0.92 | Train precision: 0.92 | Train recall: 0.92
	 Val. Loss: 0.416 |  Val. f1: 0.77 |  Val. precision: 0.79 | Val. recall: 0.75
Epoch: 06 | Epoch Time: 0m 11s
	Train Lo

####LSTM - bi - variando capas

In [ ]:
INPUT_DIM = len(TEXT.vocab)
EMBEDDING_DIM = 300  # dimensión de los embeddings. 
HIDDEN_DIM = 256  # dimensión de la capas LSTM 
OUTPUT_DIM = len(NER_TAGS.vocab)  # número de clases

N_LAYERS = 5  # número de capas.
DROPOUT = 0.3  
BIDIRECTIONAL = True

# Creamos nuestros modelos.
lstm_model = NER_RNN(INPUT_DIM, EMBEDDING_DIM, HIDDEN_DIM, OUTPUT_DIM,
                         N_LAYERS, BIDIRECTIONAL, DROPOUT, PAD_IDX)

lstm_model_name = 'LSTMbidDIM2'  # nombre que tendrá el modelo guardado...

In [ ]:
model = lstm_model
model_name = lstm_model_name
criterion = baseline_criterion

#inicializamos los pesos
model.apply(init_weights)
print(f'El modelo actual tiene {count_parameters(model):,} parámetros entrenables.')

#se determina el optimizador
optimizer = optim.Adam(model.parameters())
model = model.to(device)
criterion = criterion.to(device)

earlyStop_model(model, train_iterator, valid_iterator, optimizer, criterion)

El modelo actual tiene 12,734,080 parámetros entrenables.


/usr/local/lib/python3.7/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: <pad> seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.7/dist-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Epoch: 01 | Epoch Time: 0m 21s
	Train Loss: 0.828 | Train f1: 0.38 | Train precision: 0.57 | Train recall: 0.31
	 Val. Loss: 0.551 |  Val. f1: 0.62 |  Val. precision: 0.68 | Val. recall: 0.57
Epoch: 02 | Epoch Time: 0m 21s
	Train Loss: 0.402 | Train f1: 0.72 | Train precision: 0.77 | Train recall: 0.67
	 Val. Loss: 0.409 |  Val. f1: 0.72 |  Val. precision: 0.75 | Val. recall: 0.69
Epoch: 03 | Epoch Time: 0m 21s
	Train Loss: 0.251 | Train f1: 0.82 | Train precision: 0.83 | Train recall: 0.81
	 Val. Loss: 0.371 |  Val. f1: 0.74 |  Val. precision: 0.74 | Val. recall: 0.74
Epoch: 04 | Epoch Time: 0m 21s
	Train Loss: 0.178 | Train f1: 0.87 | Train precision: 0.87 | Train recall: 0.87
	 Val. Loss: 0.377 |  Val. f1: 0.76 |  Val. precision: 0.76 | Val. recall: 0.76
Epoch: 05 | Epoch Time: 0m 21s
	Train Loss: 0.134 | Train f1: 0.90 | Train precision: 0.90 | Train recall: 0.90
	 Val. Loss: 0.408 |  Val. f1: 0.75 |  Val. precision: 0.76 | Val. recall: 0.76
Epoch: 06 | Epoch Time: 0m 21s
	Train Lo

####LSTM - bi - variando dropout

In [ ]:
INPUT_DIM = len(TEXT.vocab)
EMBEDDING_DIM = 300  # dimensión de los embeddings. 
HIDDEN_DIM = 256  # dimensión de la capas LSTM 
OUTPUT_DIM = len(NER_TAGS.vocab)  # número de clases

N_LAYERS = 3  # número de capas.
DROPOUT = 0.5  
BIDIRECTIONAL = True

# Creamos nuestros modelos.
lstm_model = NER_RNN(INPUT_DIM, EMBEDDING_DIM, HIDDEN_DIM, OUTPUT_DIM,
                         N_LAYERS, BIDIRECTIONAL, DROPOUT, PAD_IDX)

lstm_model_name = 'LSTMbidDROP'  # nombre que tendrá el modelo guardado...

In [ ]:
model = lstm_model
model_name = lstm_model_name
criterion = baseline_criterion

#inicializamos los pesos
model.apply(init_weights)
print(f'El modelo actual tiene {count_parameters(model):,} parámetros entrenables.')

#se determina el optimizador
optimizer = optim.Adam(model.parameters())
model = model.to(device)
criterion = criterion.to(device)

earlyStop_model(model, train_iterator, valid_iterator, optimizer, criterion)

El modelo actual tiene 9,580,160 parámetros entrenables.


/usr/local/lib/python3.7/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: <pad> seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.7/dist-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Epoch: 01 | Epoch Time: 0m 13s
	Train Loss: 0.753 | Train f1: 0.45 | Train precision: 0.62 | Train recall: 0.38
	 Val. Loss: 0.465 |  Val. f1: 0.65 |  Val. precision: 0.76 | Val. recall: 0.58
Epoch: 02 | Epoch Time: 0m 13s
	Train Loss: 0.394 | Train f1: 0.72 | Train precision: 0.77 | Train recall: 0.68
	 Val. Loss: 0.379 |  Val. f1: 0.73 |  Val. precision: 0.77 | Val. recall: 0.70
Epoch: 03 | Epoch Time: 0m 13s
	Train Loss: 0.267 | Train f1: 0.81 | Train precision: 0.82 | Train recall: 0.79
	 Val. Loss: 0.368 |  Val. f1: 0.76 |  Val. precision: 0.77 | Val. recall: 0.76
Epoch: 04 | Epoch Time: 0m 13s
	Train Loss: 0.196 | Train f1: 0.86 | Train precision: 0.86 | Train recall: 0.85
	 Val. Loss: 0.349 |  Val. f1: 0.77 |  Val. precision: 0.78 | Val. recall: 0.76
Epoch: 05 | Epoch Time: 0m 13s
	Train Loss: 0.153 | Train f1: 0.89 | Train precision: 0.89 | Train recall: 0.89
	 Val. Loss: 0.373 |  Val. f1: 0.78 |  Val. precision: 0.79 | Val. recall: 0.77
Epoch: 06 | Epoch Time: 0m 13s
	Train Lo

####LSTM - bi - variando capas y dropout

In [ ]:
INPUT_DIM = len(TEXT.vocab)
EMBEDDING_DIM = 300  # dimensión de los embeddings. 
HIDDEN_DIM = 256  # dimensión de la capas LSTM 
OUTPUT_DIM = len(NER_TAGS.vocab)  # número de clases

N_LAYERS = 5  # número de capas.
DROPOUT = 0.5 
BIDIRECTIONAL = True

# Creamos nuestros modelos.
lstm_model = NER_RNN(INPUT_DIM, EMBEDDING_DIM, HIDDEN_DIM, OUTPUT_DIM,
                         N_LAYERS, BIDIRECTIONAL, DROPOUT, PAD_IDX)

lstm_model_name = 'LSTMbidCAPASDROP'  # nombre que tendrá el modelo guardado...

In [ ]:
model = lstm_model
model_name = lstm_model_name
criterion = baseline_criterion

#inicializamos los pesos
model.apply(init_weights)
print(f'El modelo actual tiene {count_parameters(model):,} parámetros entrenables.')

#se determina el optimizador
optimizer = optim.Adam(model.parameters())
model = model.to(device)
criterion = criterion.to(device)

earlyStop_model(model, train_iterator, valid_iterator, optimizer, criterion)

El modelo actual tiene 12,734,080 parámetros entrenables.


/usr/local/lib/python3.7/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: <pad> seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.7/dist-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Epoch: 01 | Epoch Time: 0m 21s
	Train Loss: 0.947 | Train f1: 0.29 | Train precision: 0.49 | Train recall: 0.23
	 Val. Loss: 0.650 |  Val. f1: 0.57 |  Val. precision: 0.73 | Val. recall: 0.47
Epoch: 02 | Epoch Time: 0m 21s
	Train Loss: 0.514 | Train f1: 0.65 | Train precision: 0.76 | Train recall: 0.58
	 Val. Loss: 0.421 |  Val. f1: 0.69 |  Val. precision: 0.80 | Val. recall: 0.62
Epoch: 03 | Epoch Time: 0m 21s
	Train Loss: 0.336 | Train f1: 0.77 | Train precision: 0.80 | Train recall: 0.74
	 Val. Loss: 0.375 |  Val. f1: 0.73 |  Val. precision: 0.75 | Val. recall: 0.71
Epoch: 04 | Epoch Time: 0m 21s
	Train Loss: 0.249 | Train f1: 0.82 | Train precision: 0.83 | Train recall: 0.81
	 Val. Loss: 0.394 |  Val. f1: 0.73 |  Val. precision: 0.70 | Val. recall: 0.76
Epoch: 05 | Epoch Time: 0m 21s
	Train Loss: 0.197 | Train f1: 0.86 | Train precision: 0.86 | Train recall: 0.85
	 Val. Loss: 0.392 |  Val. f1: 0.75 |  Val. precision: 0.75 | Val. recall: 0.76
Epoch: 06 | Epoch Time: 0m 21s
	Train Lo

####LSTM - bi - variando embedding dimension_lstm =128

In [ ]:
INPUT_DIM = len(TEXT.vocab)
EMBEDDING_DIM = 500  # dimensión de los embeddings. 
HIDDEN_DIM = 128  # dimensión de la capas LSTM 
OUTPUT_DIM = len(NER_TAGS.vocab)  # número de clases

N_LAYERS = 3  # número de capas.
DROPOUT = 0.3 
BIDIRECTIONAL = True

# Creamos nuestros modelos.
lstm_model = NER_RNN(INPUT_DIM, EMBEDDING_DIM, HIDDEN_DIM, OUTPUT_DIM,
                         N_LAYERS, BIDIRECTIONAL, DROPOUT, PAD_IDX)

lstm_model_name = 'LSTMbidEMBDIM'  # nombre que tendrá el modelo guardado...

In [ ]:
model = lstm_model
model_name = lstm_model_name
criterion = baseline_criterion

#inicializamos los pesos
model.apply(init_weights)
print(f'El modelo actual tiene {count_parameters(model):,} parámetros entrenables.')

#se determina el optimizador
optimizer = optim.Adam(model.parameters())
model = model.to(device)
criterion = criterion.to(device)

earlyStop_model(model, train_iterator, valid_iterator, optimizer, criterion)

El modelo actual tiene 10,234,232 parámetros entrenables.


/usr/local/lib/python3.7/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: <pad> seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.7/dist-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Epoch: 01 | Epoch Time: 0m 12s
	Train Loss: 0.660 | Train f1: 0.52 | Train precision: 0.64 | Train recall: 0.45
	 Val. Loss: 0.395 |  Val. f1: 0.70 |  Val. precision: 0.74 | Val. recall: 0.66
Epoch: 02 | Epoch Time: 0m 12s
	Train Loss: 0.297 | Train f1: 0.79 | Train precision: 0.82 | Train recall: 0.77
	 Val. Loss: 0.327 |  Val. f1: 0.77 |  Val. precision: 0.79 | Val. recall: 0.75
Epoch: 03 | Epoch Time: 0m 12s
	Train Loss: 0.178 | Train f1: 0.87 | Train precision: 0.88 | Train recall: 0.87
	 Val. Loss: 0.356 |  Val. f1: 0.77 |  Val. precision: 0.80 | Val. recall: 0.75
Epoch: 04 | Epoch Time: 0m 12s
	Train Loss: 0.124 | Train f1: 0.91 | Train precision: 0.91 | Train recall: 0.91
	 Val. Loss: 0.378 |  Val. f1: 0.77 |  Val. precision: 0.78 | Val. recall: 0.76
Epoch: 05 | Epoch Time: 0m 12s
	Train Loss: 0.089 | Train f1: 0.94 | Train precision: 0.93 | Train recall: 0.94
	 Val. Loss: 0.426 |  Val. f1: 0.75 |  Val. precision: 0.74 | Val. recall: 0.77
Epoch: 06 | Epoch Time: 0m 12s
	Train Lo

####LSTM - bi - variando embedding dimension_lstm =128 y dropout

In [ ]:
INPUT_DIM = len(TEXT.vocab)
EMBEDDING_DIM = 500  # dimensión de los embeddings. 
HIDDEN_DIM = 128  # dimensión de la capas LSTM 
OUTPUT_DIM = len(NER_TAGS.vocab)  # número de clases

N_LAYERS = 3  # número de capas.
DROPOUT = 0.5 
BIDIRECTIONAL = True

# Creamos nuestros modelos.
lstm_model = NER_RNN(INPUT_DIM, EMBEDDING_DIM, HIDDEN_DIM, OUTPUT_DIM,
                         N_LAYERS, BIDIRECTIONAL, DROPOUT, PAD_IDX)

lstm_model_name = 'LSTMbidEMBDIMDROP'  # nombre que tendrá el modelo guardado...

In [ ]:
model = lstm_model
model_name = lstm_model_name
criterion = baseline_criterion

#inicializamos los pesos
model.apply(init_weights)
print(f'El modelo actual tiene {count_parameters(model):,} parámetros entrenables.')

#se determina el optimizador
optimizer = optim.Adam(model.parameters())
model = model.to(device)
criterion = criterion.to(device)

earlyStop_model(model, train_iterator, valid_iterator, optimizer, criterion)

El modelo actual tiene 10,234,232 parámetros entrenables.


/usr/local/lib/python3.7/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: <pad> seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.7/dist-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Epoch: 01 | Epoch Time: 0m 12s
	Train Loss: 0.743 | Train f1: 0.47 | Train precision: 0.62 | Train recall: 0.40
	 Val. Loss: 0.450 |  Val. f1: 0.67 |  Val. precision: 0.75 | Val. recall: 0.61
Epoch: 02 | Epoch Time: 0m 12s
	Train Loss: 0.373 | Train f1: 0.74 | Train precision: 0.78 | Train recall: 0.71
	 Val. Loss: 0.392 |  Val. f1: 0.73 |  Val. precision: 0.78 | Val. recall: 0.70
Epoch: 03 | Epoch Time: 0m 12s
	Train Loss: 0.248 | Train f1: 0.82 | Train precision: 0.84 | Train recall: 0.81
	 Val. Loss: 0.365 |  Val. f1: 0.75 |  Val. precision: 0.80 | Val. recall: 0.72
Epoch: 04 | Epoch Time: 0m 12s
	Train Loss: 0.178 | Train f1: 0.87 | Train precision: 0.88 | Train recall: 0.87
	 Val. Loss: 0.364 |  Val. f1: 0.77 |  Val. precision: 0.77 | Val. recall: 0.77
Epoch: 05 | Epoch Time: 0m 12s
	Train Loss: 0.138 | Train f1: 0.90 | Train precision: 0.90 | Train recall: 0.90
	 Val. Loss: 0.399 |  Val. f1: 0.76 |  Val. precision: 0.77 | Val. recall: 0.76
Epoch: 06 | Epoch Time: 0m 12s
	Train Lo

####LSTM - bi - variando embedding version 2, dimension_lstm =128 y dropout

In [ ]:
INPUT_DIM = len(TEXT.vocab)
EMBEDDING_DIM = 800  # dimensión de los embeddings. 
HIDDEN_DIM = 128  # dimensión de la capas LSTM 
OUTPUT_DIM = len(NER_TAGS.vocab)  # número de clases

N_LAYERS = 3  # número de capas.
DROPOUT = 0.5 
BIDIRECTIONAL = True

# Creamos nuestros modelos.
lstm_model = NER_RNN(INPUT_DIM, EMBEDDING_DIM, HIDDEN_DIM, OUTPUT_DIM,
                         N_LAYERS, BIDIRECTIONAL, DROPOUT, PAD_IDX)

lstm_model_name = 'LSTMbidEMB2DIMDROP'  # nombre que tendrá el modelo guardado...

In [ ]:
model = lstm_model
model_name = lstm_model_name
criterion = baseline_criterion

#inicializamos los pesos
model.apply(init_weights)
print(f'El modelo actual tiene {count_parameters(model):,} parámetros entrenables.')

#se determina el optimizador
optimizer = optim.Adam(model.parameters())
model = model.to(device)
criterion = criterion.to(device)

earlyStop_model(model, train_iterator, valid_iterator, optimizer, criterion)

El modelo actual tiene 15,818,732 parámetros entrenables.


/usr/local/lib/python3.7/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: <pad> seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.7/dist-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Epoch: 01 | Epoch Time: 0m 13s
	Train Loss: 0.717 | Train f1: 0.49 | Train precision: 0.64 | Train recall: 0.42
	 Val. Loss: 0.433 |  Val. f1: 0.69 |  Val. precision: 0.72 | Val. recall: 0.66
Epoch: 02 | Epoch Time: 0m 13s
	Train Loss: 0.349 | Train f1: 0.76 | Train precision: 0.79 | Train recall: 0.73
	 Val. Loss: 0.357 |  Val. f1: 0.74 |  Val. precision: 0.75 | Val. recall: 0.73
Epoch: 03 | Epoch Time: 0m 13s
	Train Loss: 0.227 | Train f1: 0.84 | Train precision: 0.85 | Train recall: 0.83
	 Val. Loss: 0.352 |  Val. f1: 0.76 |  Val. precision: 0.77 | Val. recall: 0.75
Epoch: 04 | Epoch Time: 0m 13s
	Train Loss: 0.160 | Train f1: 0.89 | Train precision: 0.89 | Train recall: 0.88
	 Val. Loss: 0.393 |  Val. f1: 0.76 |  Val. precision: 0.74 | Val. recall: 0.78
Epoch: 05 | Epoch Time: 0m 14s
	Train Loss: 0.121 | Train f1: 0.91 | Train precision: 0.91 | Train recall: 0.91
	 Val. Loss: 0.439 |  Val. f1: 0.76 |  Val. precision: 0.75 | Val. recall: 0.77
Epoch: 06 | Epoch Time: 0m 13s
	Train Lo

####LSTM - bi - variando embedding version 2, dimension_lstm =128

In [ ]:
INPUT_DIM = len(TEXT.vocab)
EMBEDDING_DIM = 800  # dimensión de los embeddings. 
HIDDEN_DIM = 128  # dimensión de la capas LSTM 
OUTPUT_DIM = len(NER_TAGS.vocab)  # número de clases

N_LAYERS = 3  # número de capas.
DROPOUT = 0.3 
BIDIRECTIONAL = True

# Creamos nuestros modelos.
lstm_model = NER_RNN(INPUT_DIM, EMBEDDING_DIM, HIDDEN_DIM, OUTPUT_DIM,
                         N_LAYERS, BIDIRECTIONAL, DROPOUT, PAD_IDX)

lstm_model_name = 'LSTMbidEMB2DIM'  # nombre que tendrá el modelo guardado...

In [ ]:
model = lstm_model
model_name = lstm_model_name
criterion = baseline_criterion

#inicializamos los pesos
model.apply(init_weights)
print(f'El modelo actual tiene {count_parameters(model):,} parámetros entrenables.')

#se determina el optimizador
optimizer = optim.Adam(model.parameters())
model = model.to(device)
criterion = criterion.to(device)

earlyStop_model(model, train_iterator, valid_iterator, optimizer, criterion)

El modelo actual tiene 15,818,732 parámetros entrenables.


/usr/local/lib/python3.7/dist-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Epoch: 01 | Epoch Time: 0m 13s
	Train Loss: 0.638 | Train f1: 0.54 | Train precision: 0.66 | Train recall: 0.48
	 Val. Loss: 0.406 |  Val. f1: 0.71 |  Val. precision: 0.77 | Val. recall: 0.66
Epoch: 02 | Epoch Time: 0m 13s
	Train Loss: 0.276 | Train f1: 0.80 | Train precision: 0.82 | Train recall: 0.79
	 Val. Loss: 0.348 |  Val. f1: 0.76 |  Val. precision: 0.78 | Val. recall: 0.75
Epoch: 03 | Epoch Time: 0m 13s
	Train Loss: 0.160 | Train f1: 0.88 | Train precision: 0.89 | Train recall: 0.88
	 Val. Loss: 0.353 |  Val. f1: 0.77 |  Val. precision: 0.78 | Val. recall: 0.77
Epoch: 04 | Epoch Time: 0m 13s
	Train Loss: 0.107 | Train f1: 0.92 | Train precision: 0.92 | Train recall: 0.92
	 Val. Loss: 0.412 |  Val. f1: 0.76 |  Val. precision: 0.76 | Val. recall: 0.75
Epoch: 05 | Epoch Time: 0m 13s
	Train Loss: 0.074 | Train f1: 0.94 | Train precision: 0.94 | Train recall: 0.94
	 Val. Loss: 0.470 |  Val. f1: 0.75 |  Val. precision: 0.77 | Val. recall: 0.74
Epoch: 06 | Epoch Time: 0m 13s
	Train Lo

####LSTM - bi - variando embedding version 2, dimension_lstm =128, capas=5

In [ ]:
INPUT_DIM = len(TEXT.vocab)
EMBEDDING_DIM = 800  # dimensión de los embeddings. 
HIDDEN_DIM = 128  # dimensión de la capas LSTM 
OUTPUT_DIM = len(NER_TAGS.vocab)  # número de clases

N_LAYERS = 5  # número de capas.
DROPOUT = 0.3 
BIDIRECTIONAL = True

# Creamos nuestros modelos.
lstm_model = NER_RNN(INPUT_DIM, EMBEDDING_DIM, HIDDEN_DIM, OUTPUT_DIM,
                         N_LAYERS, BIDIRECTIONAL, DROPOUT, PAD_IDX)

lstm_model_name = 'LSTMbidEMB2DIMCAP'  # nombre que tendrá el modelo guardado...

In [ ]:
model = lstm_model
model_name = lstm_model_name
criterion = baseline_criterion

#inicializamos los pesos
model.apply(init_weights)
print(f'El modelo actual tiene {count_parameters(model):,} parámetros entrenables.')

#se determina el optimizador
optimizer = optim.Adam(model.parameters())
model = model.to(device)
criterion = criterion.to(device)

earlyStop_model(model, train_iterator, valid_iterator, optimizer, criterion)

El modelo actual tiene 16,609,260 parámetros entrenables.


/usr/local/lib/python3.7/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: <pad> seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.7/dist-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Epoch: 01 | Epoch Time: 0m 20s
	Train Loss: 0.726 | Train f1: 0.48 | Train precision: 0.61 | Train recall: 0.41
	 Val. Loss: 0.459 |  Val. f1: 0.67 |  Val. precision: 0.75 | Val. recall: 0.61
Epoch: 02 | Epoch Time: 0m 18s
	Train Loss: 0.333 | Train f1: 0.77 | Train precision: 0.80 | Train recall: 0.74
	 Val. Loss: 0.376 |  Val. f1: 0.74 |  Val. precision: 0.76 | Val. recall: 0.72
Epoch: 03 | Epoch Time: 0m 18s
	Train Loss: 0.195 | Train f1: 0.86 | Train precision: 0.86 | Train recall: 0.86
	 Val. Loss: 0.380 |  Val. f1: 0.76 |  Val. precision: 0.77 | Val. recall: 0.75
Epoch: 04 | Epoch Time: 0m 18s
	Train Loss: 0.135 | Train f1: 0.90 | Train precision: 0.90 | Train recall: 0.90
	 Val. Loss: 0.396 |  Val. f1: 0.75 |  Val. precision: 0.75 | Val. recall: 0.76
Epoch: 05 | Epoch Time: 0m 18s
	Train Loss: 0.100 | Train f1: 0.93 | Train precision: 0.93 | Train recall: 0.93
	 Val. Loss: 0.444 |  Val. f1: 0.75 |  Val. precision: 0.76 | Val. recall: 0.75
Epoch: 06 | Epoch Time: 0m 19s
	Train Lo

####LSTM - bi - variando embedding version 2, dimension_lstm =128, capas=5, DROPOUT

In [ ]:
INPUT_DIM = len(TEXT.vocab)
EMBEDDING_DIM = 800  # dimensión de los embeddings. 
HIDDEN_DIM = 128  # dimensión de la capas LSTM 
OUTPUT_DIM = len(NER_TAGS.vocab)  # número de clases

N_LAYERS = 5  # número de capas.
DROPOUT = 0.3 
BIDIRECTIONAL = True

# Creamos nuestros modelos.
lstm_model = NER_RNN(INPUT_DIM, EMBEDDING_DIM, HIDDEN_DIM, OUTPUT_DIM,
                         N_LAYERS, BIDIRECTIONAL, DROPOUT, PAD_IDX)

lstm_model_name = 'LSTMbidEMB2DIMCAPDROP'  # nombre que tendrá el modelo guardado...

In [ ]:
model = lstm_model
model_name = lstm_model_name
criterion = baseline_criterion

#inicializamos los pesos
model.apply(init_weights)
print(f'El modelo actual tiene {count_parameters(model):,} parámetros entrenables.')

#se determina el optimizador
optimizer = optim.Adam(model.parameters())
model = model.to(device)
criterion = criterion.to(device)

earlyStop_model(model, train_iterator, valid_iterator, optimizer, criterion)

El modelo actual tiene 16,609,260 parámetros entrenables.


/usr/local/lib/python3.7/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: <pad> seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.7/dist-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Epoch: 01 | Epoch Time: 0m 18s
	Train Loss: 0.709 | Train f1: 0.50 | Train precision: 0.64 | Train recall: 0.43
	 Val. Loss: 0.438 |  Val. f1: 0.70 |  Val. precision: 0.74 | Val. recall: 0.66
Epoch: 02 | Epoch Time: 0m 18s
	Train Loss: 0.325 | Train f1: 0.78 | Train precision: 0.80 | Train recall: 0.75
	 Val. Loss: 0.354 |  Val. f1: 0.74 |  Val. precision: 0.75 | Val. recall: 0.74
Epoch: 03 | Epoch Time: 0m 18s
	Train Loss: 0.197 | Train f1: 0.86 | Train precision: 0.86 | Train recall: 0.86
	 Val. Loss: 0.396 |  Val. f1: 0.76 |  Val. precision: 0.79 | Val. recall: 0.73
Epoch: 04 | Epoch Time: 0m 18s
	Train Loss: 0.136 | Train f1: 0.90 | Train precision: 0.90 | Train recall: 0.90
	 Val. Loss: 0.419 |  Val. f1: 0.75 |  Val. precision: 0.75 | Val. recall: 0.75
Epoch: 05 | Epoch Time: 0m 19s
	Train Loss: 0.104 | Train f1: 0.92 | Train precision: 0.92 | Train recall: 0.93
	 Val. Loss: 0.469 |  Val. f1: 0.77 |  Val. precision: 0.79 | Val. recall: 0.75
Epoch: 06 | Epoch Time: 0m 18s
	Train Lo

### **Predecir datos para la competencia**

Ahora, a partir de los datos de **test** y nuestro modelo entrenado, vamos a predecir las etiquetas que serán evaluadas en la competencia.

In [ ]:
def predict_labels(model, iterator, criterion, fields=fields):

    # Extraemos los vocabularios.
    text_field = fields[0][1]
    nertags_field = fields[1][1]
    tags_vocab = nertags_field.vocab.itos
    words_vocab = text_field.vocab.itos

    model.eval()

    predictions = []

    with torch.no_grad():

        for batch in iterator:

            text_batch = batch.text
            text_batch = torch.transpose(text_batch, 0, 1).tolist()

            # Predecir los tags de las sentences del batch
            predictions_batch = model(batch.text)
            predictions_batch = torch.transpose(predictions_batch, 0, 1)

            # por cada oración predicha:
            for sentence, sentence_prediction in zip(text_batch,
                                                     predictions_batch):
                for word_idx, word_predictions in zip(sentence,
                                                      sentence_prediction):
                    # Obtener el indice del tag con la probabilidad mas alta.
                    argmax_index = word_predictions.topk(1)[1]

                    current_tag = tags_vocab[argmax_index]
                    # Obtenemos la palabra
                    current_word = words_vocab[word_idx]

                    if current_word != '<pad>':
                        predictions.append([current_word, current_tag])
                predictions.append(['EOS', 'EOS'])


    return predictions


predictions = predict_labels(model, test_iterator, criterion)

In [ ]:
predictions

### **Generar el archivo para la submission**

No hay problema si aparecen unk en la salida. Estos no son relevantes para evaluarlos, usamos solo los tags.

In [ ]:
import os, shutil

if (os.path.isfile('./predictions.zip')):
    os.remove('./predictions.zip')

if (not os.path.isdir('./predictions')):
    os.mkdir('./predictions')

else:
    # Eliminar predicciones anteriores:
    shutil.rmtree('./predictions')
    os.mkdir('./predictions')

f = open('predictions/predictions.txt', 'w')
for i, (word, tag) in enumerate(predictions[:-1]):
    if word=='EOS' and tag=='EOS': f.write('\n')
    else: 
      if i == len(predictions[:-1])-1:
        f.write(word + ' ' + tag)
      else: f.write(word + ' ' + tag + '\n')

f.close()

a = shutil.make_archive('predictions', 'zip', './predictions')

##**Resultados**

|No.| Modelo                       |Epochs|Loss | f1 |precision|recall|
|---|------------------------------|------|-----|----|---------|------|
| 0 |LSTM                          |  13  |0.381|0.73|  0.73   | 0.72 |
| 1 |Elman + earlyStop(ES)         |  20  |0.468|0.69|  0.71   | 0.66 |
| 2 |GRU + ES                      |  12  |0.399|0.72|  0.78   | 0.67 |
| 3 |LSTM Bidireccional (BI) + ES  |  12  |0.332|0.75|  0.79   | 0.72 |
| 4 |Elman BI + ES                 |  10  |0.716|0.52|  0.62   | 0.45 |
| 5 |GRU BI + ES                   |  13  |0.366|0.75|  0.78   | 0.73 |
| 6 |LSTM BI + ES + EMBEDDING (EMB)|  12  |0.340|0.76|  0.78   | 0.75 |
| 7 |LSTM BI + ES + DIM=512 (D1)   |  13  |0.368|0.76|  0.79   | 0.74 |
| 8 |LSTM BI + ES + DIM=128 (D2)   |  13  |0.347|0.76|  0.79   | 0.74 |
| 9 |LSTM BI + ES + CAPAS=5 (CA)   |  13  |0.371|0.74|  0.74   | 0.74 |
| 10|LSTM BI + ES + DROPOUT=0.5(DR)|  14  |0.349|0.77|  0.78   | 0.76 |
| 11|LSTM BI + ES + CA + DR        |  13  |0.375|0.73|  0.75   | 0.71 |
| 12|LSTM BI + ES + EMB + D2       |  12  |0.327|0.77|  0.79   | 0.75 |
| 13|LSTM BI + ES + EMB + D2 + DR  |  14  |0.364|0.77|  0.77   | 0.77 |
| 14|LSTM BI + ES + EMB2 + D2 + DR |  13  |0.352|0.76|  0.77   | 0.75 |
| 15|LSTM BI + ES + EMB2 + D2      |  12  |0.344|0.78|  0.80   | 0.76 |
| 16|LSTM BI + ES + EMB2 + D2 + CA |  14  |0.376|0.74|  0.76   | 0.72 |
| 17|LSTM BI + ES + EMB2 + D2 + CA + DR|  12  |0.354|0.74|  0.75   | 0.74 |



Observamos de los resultados mostrados previamente, que los modelos bidireccionales son consistentemente mejores que los modelos sin bidireccionalidad. Ahora bien, dentro de estos, el modelo con mejores resultados es la LSTM. Al probar distintas arquitecturas de LSTM vemos que el modelo que obtiene la menor loss es el modelo número 12 que mezcla un cambio en el embedding y un menor numero de hidden_layers. No obstante, el modelo con mejor métrica de desempeño en la predicción es el modelo 15 que incorpora otra versión del cambio de embedding. Ahora bien, la mejora es muy menor para la cantidad de parametros extra que se crean, por lo que se concluye que el mejor modelo creado es el número 12.

## **Conclusiones**



Los resultados en el dataset de validación fueron razonablemente buenos. No obstante, se observó mucho overfitting en los modelos planteados. Lo cual se trasujo en un bajo número de epocas de entrenamiento. Respecto a las métricas, se tienen resultados razonables, pero que al extrapolarse al dataset de testeo, fueron inferiores a las esperadas.

Para poder mejorar los modelos, se propone en primer lugar, variar el learning rate a valores menores, pues una opcion posible es que las redes estén cayendo dentro de un mínimo local que propicie el overfitting. Pasando a metodologías posibles a aplicar, se tiene el uso de transformers, otros embeddings para generar los inputs de las redes neuronales o realizar el ejercicio de distintas arquitecturas sobre modelo GRU.